## 開催日を取得
https://race.netkeiba.com/top/calendar.html?year=2024&month=1 から開催日をスクレイピング  
　　　　　　　　　　↓  
例えば https://race.netkeiba.com/top/race_list.html?kaisai_date=20240901 のようにdateの部分に開催日を入れて、  
ここから各race_idを取得していく

In [8]:
import pandas as pd
from modules.constants import local_paths, master
import os
from modules import prepare_html, scraping
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import ast
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import datetime
from tqdm import tqdm
import time
import re




def get_race_date_list(start: str, end: str):
  dstart = datetime.datetime.strptime(start, '%Y-%m')
  dend = datetime.datetime.strptime(end, '%Y-%m')
  start_year = dstart.year
  end_year = dend.year
  start_month = dstart.month
  end_month = dend.month

  race_date_list = []
  for year in range(start_year, end_year+1):
    for month in tqdm(range(start_month, end_month+1)):
      url = f'https://race.netkeiba.com/top/calendar.html?year={year}&month={month}'
      html = urlopen(url)
      soup = BeautifulSoup(html, "html.parser")
      time.sleep(1)
      # リンクのある部分（開催日）をスクレイピング
      a = soup.find_all("a", attrs={"href": re.compile(r'../top/race_list\.html.kaisai_date=[0-9]+')})

      # 開催日のリスト化
      for tag in a:
        race_date = re.search(r'[0-9]+', tag["href"]).group()
        race_date_list.append(race_date)
  
  return race_date_list

  


In [145]:
print(get_race_date_list('2011-1', '2011-12'))

100%|██████████| 2/2 [00:02<00:00,  1.19s/it]

['20110105', '20110108', '20110109', '20110110', '20110115', '20110116', '20110122', '20110123', '20110129', '20110130', '20110205', '20110206', '20110212', '20110213', '20110214', '20110219', '20110220', '20110226', '20110227']


In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re
from tqdm import tqdm
def get_race_id_list(race_date_list: list[str]) -> list[str]:
  options = Options()
  options.add_argument("--headless")
  driver_path = ChromeDriverManager().install()

  race_id_list = []
  with webdriver.Chrome(service=Service(driver_path), options=options) as driver:
    for race_date in tqdm(race_date_list):
      url = f'https://race.netkeiba.com/top/race_list.html?kaisai_date={race_date}'
      driver.get(url)
      time.sleep(1)
      li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
      for li in li_list:
        href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
        race_id = re.findall(r'race_id=(\d{12})', href)[0]
        race_id_list.append(race_id)
  
    return race_id_list


In [137]:
race_date_list = get_race_date_list('2023-1', '2023-12')
race_date_list

['20110105',
 '20110108',
 '20110109',
 '20110110',
 '20110115',
 '20110116',
 '20110122',
 '20110123',
 '20110129',
 '20110130',
 '20110205',
 '20110206',
 '20110212',
 '20110213',
 '20110214',
 '20110219',
 '20110220',
 '20110226',
 '20110227']

In [140]:
get_race_id_list(race_date_list)

100%|██████████| 19/19 [01:35<00:00,  5.02s/it]


['201106010101',
 '201106010102',
 '201106010103',
 '201106010104',
 '201106010105',
 '201106010106',
 '201106010107',
 '201106010108',
 '201106010109',
 '201106010110',
 '201106010111',
 '201106010112',
 '201108010101',
 '201108010102',
 '201108010103',
 '201108010104',
 '201108010105',
 '201108010106',
 '201108010107',
 '201108010108',
 '201108010109',
 '201108010110',
 '201108010111',
 '201108010112',
 '201106010201',
 '201106010202',
 '201106010203',
 '201106010204',
 '201106010205',
 '201106010206',
 '201106010207',
 '201106010208',
 '201106010209',
 '201106010210',
 '201106010211',
 '201106010212',
 '201108010201',
 '201108010202',
 '201108010203',
 '201108010204',
 '201108010205',
 '201108010206',
 '201108010207',
 '201108010208',
 '201108010209',
 '201108010210',
 '201108010211',
 '201108010212',
 '201106010301',
 '201106010302',
 '201106010303',
 '201106010304',
 '201106010305',
 '201106010306',
 '201106010307',
 '201106010308',
 '201106010309',
 '201106010310',
 '201106010311

## レース結果の取得
race_id_listをもとに、例えば https://race.netkeiba.com/race/result.html?race_id=202404030801&rf=race_list のようなurlからレース結果を取得する

In [1]:
import pandas as pd
import pickle
import os
from urllib.request import urlopen
import time
from tqdm import tqdm

from modules.constants import url_paths, local_paths

def get_html_race(race_id_list: list[str], save_dir: str = local_paths.HTML_RACE_DIR) -> list[str]:
  html_path_list = []
  for race_id in tqdm(race_id_list):
    filepath = os.path.join(save_dir, f"{race_id}.bin")
    html_path_list.append(filepath)

    if os.path.isfile(filepath):
      print(f"skipped: {race_id}")
    else:
      url =  os.path.join(url_paths.RACE_URL, race_id)
      html = urlopen(url).read()
      time.sleep(1)

      with open(filepath, "wb") as f:
        f.write(html) 
      
  return html_path_list


In [10]:
race_id_list = pd.read_pickle(local_paths.RACE_ID_LIST_DIR)
html_paths_race = get_html_race(race_id_list)
html_paths_race

100%|██████████| 3456/3456 [00:00<00:00, 20660.01it/s]

skipped: 202306010101
skipped: 202306010102
skipped: 202306010103
skipped: 202306010104
skipped: 202306010105
skipped: 202306010106
skipped: 202306010107
skipped: 202306010108
skipped: 202306010109
skipped: 202306010110
skipped: 202306010111
skipped: 202306010112
skipped: 202307010101
skipped: 202307010102
skipped: 202307010103
skipped: 202307010104
skipped: 202307010105
skipped: 202307010106
skipped: 202307010107
skipped: 202307010108
skipped: 202307010109
skipped: 202307010110
skipped: 202307010111
skipped: 202307010112
skipped: 202306010201
skipped: 202306010202
skipped: 202306010203
skipped: 202306010204
skipped: 202306010205
skipped: 202306010206
skipped: 202306010207
skipped: 202306010208
skipped: 202306010209
skipped: 202306010210
skipped: 202306010211
skipped: 202306010212
skipped: 202307010201
skipped: 202307010202
skipped: 202307010203
skipped: 202307010204
skipped: 202307010205
skipped: 202307010206
skipped: 202307010207
skipped: 202307010208
skipped: 202307010209
skipped: 2

['c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010102.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010103.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010104.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010105.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010106.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010107.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010108.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010109.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010110.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010111.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010112.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202307010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202307010102.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\

## レース結果データの加工(raw_data化)

In [24]:
def create_results(html_paths_race: list[str]) -> pd.DataFrame:
  dfs = {}
  for html_path in tqdm(html_paths_race):
    with open(html_path, "rb") as f:
      race_id = re.search(r'\d{12}', html_path).group()
      html = f.read()
      df = pd.read_html(html)[0]
      df.index = [race_id] * len(df)
      dfs[race_id] = df
  concat_df = pd.concat(dfs.values())
  concat_df.index.name = "race_id"
  return concat_df

In [25]:
create_results(html_paths_race[:5])

100%|██████████| 5/5 [00:00<00:00, 39.51it/s]


,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202306010101,1,7,14,シュバルツガイスト,牡3,56,武豊,1:12.6,NaN,1.8,1,486(-4),[東] 国枝栄
202306010101,2,3,5,ジェイケイファイン,牡3,56,内田博幸,1:12.9,1.3/4,7.4,3,472(-8),[東] 奥平雅士
202306010101,3,1,1,レディアス,牝3,54,津村明秀,1:12.9,クビ,36.1,6,428(+6),[東] 鈴木慎太
202306010101,4,8,15,ハローマイラブリー,牝3,54,三浦皇成,1:13.4,3,4.2,2,536(0),[東] 伊坂重信
202306010101,5,5,9,ベアカワイコチャン,牝3,54,松岡正海,1:13.9,3,7.9,4,478(+10),[東] 杉浦宏昭
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202306010105,12,5,10,ニシノカナン,牡3,56,勝浦正樹,1:35.7,1/2,79.4,13,506(+8),[東] 杉浦宏昭
202306010105,13,4,7,ニシノスナイパー,牡3,56,武藤雅,1:36.2,3,72.5,12,464(+2),[東] 手塚貴久
202306010105,14,6,12,ロマンスオーサー,牡3,56,バシュロ,1:36.4,1.1/2,31.3,9,496(+2),[東] 加藤征弘


In [49]:
results_path = os.path.join(local_paths.RAW_DIR, "results.csv")
results = pd.read_csv(results_path, index_col=0, encoding='utf8', sep='\t')

In [4]:
results.columns = results.columns.str.replace(" ", "")

In [5]:
results.columns

Index(['着順', '枠番', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人気',
       '馬体重', '調教師', 'horse_id', 'jockey_id', 'trainer_id', 'owner_id'],
      dtype='object')

In [57]:
results

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id,trainer_id,owner_id,rank
0,202306010101,1,7,14,シュバルツガイスト,牡3,56.0,武豊,1:12.6,NaN,1.8,1.0,486(-4),[東] 国枝栄,2020103575,2020103575,2020103575,2020103575,1.0
1,202306010101,2,3,5,ジェイケイファイン,牡3,56.0,内田博幸,1:12.9,1.3/4,7.4,3.0,472(-8),[東] 奥平雅士,2020107073,2020107073,2020107073,2020107073,2.0
2,202306010101,3,1,1,レディアス,牝3,54.0,津村明秀,1:12.9,クビ,36.1,6.0,428(+6),[東] 鈴木慎太,2020102562,2020102562,2020102562,2020102562,3.0
3,202306010101,4,8,15,ハローマイラブリー,牝3,54.0,三浦皇成,1:13.4,3,4.2,2.0,536(0),[東] 伊坂重信,2020106345,2020106345,2020106345,2020106345,4.0
4,202306010101,5,5,9,ベアカワイコチャン,牝3,54.0,松岡正海,1:13.9,3,7.9,4.0,478(+10),[東] 杉浦宏昭,2020100039,2020100039,2020100039,2020100039,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47667,202309050912,12,6,11,エグレムニ,牡6,58.0,松若風馬,1:09.5,3/4,172.3,13.0,512(+4),[西] 加用正,2017104873,2017104873,2017104873,2017104873,12.0
47668,202309050912,13,8,15,ハギノメーテル,牝4,56.0,藤懸貴志,1:09.5,1/2,94.7,12.0,498(+10),[西] 寺島良,2019100653,2019100653,2019100653,2019100653,13.0
47669,202309050912,14,4,7,アビエルト,牡6,58.0,和田竜二,1:09.6,1/2,75.0,10.0,508(+2),[西] 杉山佳明,2017106240,2017106240,2017106240,2017106240,14.0
47670,202309050912,15,8,16,クムシラコ,牡5,58.0,荻野琢真,1:09.8,3/4,368.3,15.0,472(-4),[東] 石毛善彦,2018103205,2018103205,2018103205,2018103205,15.0


### 馬の戦績データについても加工(raw_data化)

In [65]:
horse_id_list = results['horse_id']
html_paths_horse = prepare_html.get_html_horse(horse_id_list)
with open(html_paths_horse[0], "rb") as f:
  html = f.read()
  df = pd.read_html(html)[3]

  5%|▍         | 2185/47273 [00:00<00:02, 19149.87it/s]

skipped: 2020103575
skipped: 2020107073
skipped: 2020102562
skipped: 2020106345
skipped: 2020100039
skipped: 2020104817
skipped: 2020104295
skipped: 2020105491
skipped: 2020104358
skipped: 2020105324
skipped: 2020103960
skipped: 2020102110
skipped: 2020102145
skipped: 2020101415
skipped: 2020106714
skipped: 2020100424
skipped: 2020104020
skipped: 2020105860
skipped: 2020100382
skipped: 2020103088
skipped: 2020110020
skipped: 2020105770
skipped: 2020105381
skipped: 2020110035
skipped: 2020106354
skipped: 2020105061
skipped: 2020101465
skipped: 2020106013
skipped: 2020105675
skipped: 2020100066
skipped: 2020102901
skipped: 2020103763
skipped: 2020103622
skipped: 2020100572
skipped: 2020104541
skipped: 2020106690
skipped: 2020102538
skipped: 2020105712
skipped: 2020102338
skipped: 2020100417
skipped: 2020102698
skipped: 2020109102
skipped: 2020102722
skipped: 2020103706
skipped: 2020106259
skipped: 2020103824
skipped: 2020100936
skipped: 2017104947
skipped: 2018100186
skipped: 2019100682


 15%|█▌        | 7131/47273 [00:00<00:01, 23164.74it/s]

skipped: 2018104420
skipped: 2017106451
skipped: 2019105731
skipped: 2020106680
skipped: 2020106885
skipped: 2020103491
skipped: 2020100426
skipped: 2020101551
skipped: 2020102340
skipped: 2020104986
skipped: 2020103160
skipped: 2020106707
skipped: 2019101661
skipped: 2019100635
skipped: 2019100284
skipped: 2017104884
skipped: 2019101835
skipped: 2019103219
skipped: 2019103278
skipped: 2019104239
skipped: 2019105186
skipped: 2019100759
skipped: 2017104266
skipped: 2018103280
skipped: 2018104723
skipped: 2019102220
skipped: 2019105485
skipped: 2019106935
skipped: 2019101635
skipped: 2018106739
skipped: 2018104162
skipped: 2018103603
skipped: 2019105908
skipped: 2019105341
skipped: 2018103496
skipped: 2019105120
skipped: 2019103158
skipped: 2017106399
skipped: 2019109123
skipped: 2019105367
skipped: 2020103284
skipped: 2020102132
skipped: 2020101217
skipped: 2020101262
skipped: 2020100909
skipped: 2020106737
skipped: 2020103962
skipped: 2020106343
skipped: 2020104856
skipped: 2020102049


 26%|██▌       | 12191/47273 [00:00<00:01, 22681.28it/s]

...

skipped: 2018103563
skipped: 2013101904
skipped: 2019105392
skipped: 2014102394
skipped: 2018100220
skipped: 2015105399
skipped: 2018105731
skipped: 2017104767
skipped: 2018100905
skipped: 2016102156
skipped: 2017104690
skipped: 2021103323
skipped: 2021105122
skipped: 2021100899
skipped: 2021105054
skipped: 2021105057
skipped: 2021102670
skipped: 2021102154
skipped: 2021105175
skipped: 2021106395
skipped: 2021100359
skipped: 2020100254
skipped: 2018105301
skipped: 2019105370
skipped: 2019103885
skipped: 2018101626
skipped: 2019100549
skipped: 2019100788
skipped: 2019104769
skipped: 2019105421
skipped: 2017102171
skipped: 2020106654
skipped: 2018105360
skipped: 2019100541
skipped: 2019100242
skipped: 2019101633
skipped: 2017106572
skipped: 2017101534
skipped: 2017105282
skipped: 2017100167
skipped: 2017104552
skipped: 2018104121
skipped: 2018103173
skipped: 2015104699
skipped: 2017103616
skipped: 2020102688
skipped: 2020101139
skipped: 2020106908
skipped: 2019100072
skipped: 2018101742


 90%|█████████ | 42600/47273 [00:01<00:00, 23770.35it/s]

skipped: 2020106385
skipped: 2018106606
skipped: 2020103612
skipped: 2020100818
skipped: 2020106782
skipped: 2019104316
skipped: 2020105813
skipped: 2019104135
skipped: 2018105102
skipped: 2019105142
skipped: 2020100714
skipped: 2020103570
skipped: 2020100552
skipped: 2020102645
skipped: 2019101340
skipped: 2020106488
skipped: 2020102762
skipped: 2019105492
skipped: 2020101343
skipped: 2020103304
skipped: 2020104266
skipped: 2020103641
skipped: 2019104825
skipped: 2018102173
skipped: 2019101882
skipped: 2019105450
skipped: 2018104261
skipped: 2018101660
skipped: 2018104225
skipped: 2018104697
skipped: 2019100787
skipped: 2018105321
skipped: 2018104591
skipped: 2019105197
skipped: 2019102628
skipped: 2020103262
skipped: 2019100547
skipped: 2017104988
skipped: 2019100215
skipped: 2018102172
skipped: 2017105588
skipped: 2016102362
skipped: 2018105088
skipped: 2016105206
skipped: 2019103507
skipped: 2018105448
skipped: 2018102195
skipped: 2017102011
skipped: 2019104589
skipped: 2015105868


100%|██████████| 47273/47273 [00:01<00:00, 23655.45it/s]

skipped: 2021102991
skipped: 2021106916
skipped: 2021101131
skipped: 2021100838
skipped: 2021106988
skipped: 2021103270
skipped: 2021105241
skipped: 2021105065
skipped: 2021106005
skipped: 2021103372
skipped: 2021104518
skipped: 2021102051
skipped: 2021103827
skipped: 2019106099
skipped: 2020105167
skipped: 2020100702
skipped: 2020104751
skipped: 2019106028
skipped: 2018100813
skipped: 2018103117
skipped: 2019103965
skipped: 2020103833
skipped: 2020105145
skipped: 2016100930
skipped: 2019104119
skipped: 2019104080
skipped: 2019100617
skipped: 2019103506
skipped: 2019103036
skipped: 2020102658
skipped: 2020102343
skipped: 2020110069
skipped: 2020105107
skipped: 2020102664
skipped: 2020110092
skipped: 2021110132
skipped: 2021107058
skipped: 2021105054
skipped: 2021105358
skipped: 2021104140
skipped: 2021109123
skipped: 2021100782
skipped: 2021104278
skipped: 2021105753
skipped: 2021106854
skipped: 2018102969
skipped: 2019101005
skipped: 2019101975
skipped: 2020102776
skipped: 2018100478


In [66]:
df

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2024/08/31,2中京7,小雨,10,鳥羽特別(2勝クラス),NaN,15,3,5,32.6,...,4.6,**,10-10,34.7-36.8,40.5,496(-8),NaN,NaN,カズゴルティス,NaN
1,2024/07/27,2新潟1,曇,12,出雲崎特別(2勝クラス),NaN,15,6,10,8.0,...,0.9,**,4-4,34.2-36.7,37.3,504(+10),NaN,NaN,エランティス,NaN
2,2024/05/19,1新潟8,晴,10,火打山特別(2勝クラス),NaN,15,5,9,13.0,...,0.6,**,6-6,33.6-37.5,37.1,494(-6),NaN,NaN,シラキヌ,230.0
3,2024/05/05,1新潟4,晴,12,4歳以上1勝クラス,NaN,13,4,4,4.5,...,-0.2,**,7-7,35.2-37.6,36.9,500(+2),NaN,NaN,(セキテイオー),800.0
4,2024/02/25,1阪神2,小雨,12,4歳以上1勝クラス,NaN,15,7,13,8.6,...,0.9,**,8-10,34.6-36.4,36.3,498(0),NaN,NaN,サクハル,NaN
5,2023/12/16,5中山5,晴,12,3歳以上1勝クラス,NaN,16,4,7,4.1,...,1.4,**,2-3,34.7-36.5,37.7,498(+4),NaN,NaN,ヤングワールド,NaN
6,2023/09/24,4中山7,晴,8,3歳以上1勝クラス,NaN,16,1,2,2.9,...,1.2,**,4-4,33.2-38.2,38.9,494(-2),NaN,NaN,トモジャミ,80.0
7,2023/08/12,3新潟1,晴,12,3歳以上1勝クラス,NaN,15,8,15,5.0,...,0.6,**,3-3,34.6-36.5,36.7,496(-4),NaN,NaN,キュートヘスティア,320.0
8,2023/07/22,2福島7,晴,12,3歳以上1勝クラス,NaN,16,8,15,8.5,...,0.9,**,10-8,30.8-37.5,37.2,500(+8),NaN,NaN,シアージスト,120.0
9,2023/05/06,2東京5,晴,7,3歳1勝クラス,NaN,16,2,3,53.6,...,2.1,**,11-9,34.4-34.9,36.0,492(-4),NaN,NaN,リサリサ,NaN


## rawdataを使えるデータに前処理(preprocessed_data化)

In [54]:
results['rank'] = pd.to_numeric(results['着順'], errors="coerce") # 数値型に変換（欠損値をnanに）
results['rank'].value_counts(dropna=False)

rank
3.0     3460
1.0     3459
2.0     3456
4.0     3456
5.0     3456
6.0     3443
7.0     3424
8.0     3367
9.0     3259
10.0    3101
11.0    2895
12.0    2637
13.0    2342
14.0    2047
15.0    1708
16.0    1279
NaN      399
17.0     280
18.0     204
Name: count, dtype: int64

In [55]:
results.dropna(subset=['rank'], inplace=True)

In [56]:
results['rank'].value_counts(dropna=False)

rank
3.0     3460
1.0     3459
2.0     3456
4.0     3456
5.0     3456
6.0     3443
7.0     3424
8.0     3367
9.0     3259
10.0    3101
11.0    2895
12.0    2637
13.0    2342
14.0    2047
15.0    1708
16.0    1279
17.0     280
18.0     204
Name: count, dtype: int64

In [18]:
results['枠番'].astype(int)

race_id
202306010101    7
202306010101    3
202306010101    1
202306010101    8
202306010101    5
               ..
202309050912    6
202309050912    8
202309050912    4
202309050912    8
202309050912    7
Name: 枠番, Length: 47273, dtype: int64

In [19]:
results['馬番'].astype(int)

race_id
202306010101    14
202306010101     5
202306010101     1
202306010101    15
202306010101     9
                ..
202309050912    11
202309050912    15
202309050912     7
202309050912    16
202309050912    14
Name: 馬番, Length: 47273, dtype: int64

In [22]:
results['性齢'].str[0].value_counts()

性齢
牡    25038
牝    19922
セ     2313
Name: count, dtype: int64

In [24]:
sex_mapping = {"牡": 0, "牝": 1, "セ": 2}

In [25]:
results['性齢'].str[0].map(sex_mapping).value_counts()

性齢
0    25038
1    19922
2     2313
Name: count, dtype: int64

In [28]:
results['性齢'].str[1:].astype(int)

race_id
202306010101    3
202306010101    3
202306010101    3
202306010101    3
202306010101    3
               ..
202309050912    6
202309050912    4
202309050912    6
202309050912    5
202309050912    6
Name: 性齢, Length: 47273, dtype: int64

In [30]:
results['単勝'].astype(float)

race_id
202306010101      1.8
202306010101      7.4
202306010101     36.1
202306010101      4.2
202306010101      7.9
                ...  
202309050912    172.3
202309050912     94.7
202309050912     75.0
202309050912    368.3
202309050912    347.8
Name: 単勝, Length: 47273, dtype: float64

In [31]:
results['人気'].astype(int)

race_id
202306010101     1
202306010101     3
202306010101     6
202306010101     2
202306010101     4
                ..
202309050912    13
202309050912    12
202309050912    10
202309050912    15
202309050912    14
Name: 人気, Length: 47273, dtype: int64

In [35]:
results['馬体重']

race_id
202306010101     486(-4)
202306010101     472(-8)
202306010101     428(+6)
202306010101      536(0)
202306010101    478(+10)
                  ...   
202309050912     512(+4)
202309050912    498(+10)
202309050912     508(+2)
202309050912     472(-4)
202309050912    490(+14)
Name: 馬体重, Length: 47273, dtype: object

In [34]:
results['馬体重'].str.extract(r'(\d+)').astype(int)

,0
race_id,
202306010101,486
202306010101,472
202306010101,428
202306010101,536
202306010101,478
...,...
202309050912,512
202309050912,498
202309050912,508


In [39]:
results['weight_diff'] = results['馬体重'].str.extract(r'\((.+)\)').astype(int)

In [40]:
results['斤量']

race_id
202306010101    56.0
202306010101    56.0
202306010101    54.0
202306010101    54.0
202306010101    54.0
                ... 
202309050912    58.0
202309050912    56.0
202309050912    58.0
202309050912    58.0
202309050912    58.0
Name: 斤量, Length: 47273, dtype: float64

In [46]:
results

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id,trainer_id,owner_id
race_id,,,,,,,,,,,,,,,,,
202306010101,1,7,14,シュバルツガイスト,牡3,56.0,武豊,1:12.6,NaN,1.8,1.0,486(-4),[東] 国枝栄,2020103575,2020103575,2020103575,2020103575
202306010101,2,3,5,ジェイケイファイン,牡3,56.0,内田博幸,1:12.9,1.3/4,7.4,3.0,472(-8),[東] 奥平雅士,2020107073,2020107073,2020107073,2020107073
202306010101,3,1,1,レディアス,牝3,54.0,津村明秀,1:12.9,クビ,36.1,6.0,428(+6),[東] 鈴木慎太,2020102562,2020102562,2020102562,2020102562
202306010101,4,8,15,ハローマイラブリー,牝3,54.0,三浦皇成,1:13.4,3,4.2,2.0,536(0),[東] 伊坂重信,2020106345,2020106345,2020106345,2020106345
202306010101,5,5,9,ベアカワイコチャン,牝3,54.0,松岡正海,1:13.9,3,7.9,4.0,478(+10),[東] 杉浦宏昭,2020100039,2020100039,2020100039,2020100039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202309050912,12,6,11,エグレムニ,牡6,58.0,松若風馬,1:09.5,3/4,172.3,13.0,512(+4),[西] 加用正,2017104873,2017104873,2017104873,2017104873
202309050912,13,8,15,ハギノメーテル,牝4,56.0,藤懸貴志,1:09.5,1/2,94.7,12.0,498(+10),[西] 寺島良,2019100653,2019100653,2019100653,2019100653
202309050912,14,4,7,アビエルト,牡6,58.0,和田竜二,1:09.6,1/2,75.0,10.0,508(+2),[西] 杉山佳明,2017106240,2017106240,2017106240,2017106240


In [47]:
results.reset_index(inplace=True)


In [48]:
results

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id,trainer_id,owner_id
0,202306010101,1,7,14,シュバルツガイスト,牡3,56.0,武豊,1:12.6,NaN,1.8,1.0,486(-4),[東] 国枝栄,2020103575,2020103575,2020103575,2020103575
1,202306010101,2,3,5,ジェイケイファイン,牡3,56.0,内田博幸,1:12.9,1.3/4,7.4,3.0,472(-8),[東] 奥平雅士,2020107073,2020107073,2020107073,2020107073
2,202306010101,3,1,1,レディアス,牝3,54.0,津村明秀,1:12.9,クビ,36.1,6.0,428(+6),[東] 鈴木慎太,2020102562,2020102562,2020102562,2020102562
3,202306010101,4,8,15,ハローマイラブリー,牝3,54.0,三浦皇成,1:13.4,3,4.2,2.0,536(0),[東] 伊坂重信,2020106345,2020106345,2020106345,2020106345
4,202306010101,5,5,9,ベアカワイコチャン,牝3,54.0,松岡正海,1:13.9,3,7.9,4.0,478(+10),[東] 杉浦宏昭,2020100039,2020100039,2020100039,2020100039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47667,202309050912,12,6,11,エグレムニ,牡6,58.0,松若風馬,1:09.5,3/4,172.3,13.0,512(+4),[西] 加用正,2017104873,2017104873,2017104873,2017104873
47668,202309050912,13,8,15,ハギノメーテル,牝4,56.0,藤懸貴志,1:09.5,1/2,94.7,12.0,498(+10),[西] 寺島良,2019100653,2019100653,2019100653,2019100653
47669,202309050912,14,4,7,アビエルト,牡6,58.0,和田竜二,1:09.6,1/2,75.0,10.0,508(+2),[西] 杉山佳明,2017106240,2017106240,2017106240,2017106240
47670,202309050912,15,8,16,クムシラコ,牡5,58.0,荻野琢真,1:09.8,3/4,368.3,15.0,472(-4),[東] 石毛善彦,2018103205,2018103205,2018103205,2018103205


In [4]:
horse_results = pd.read_csv(os.path.join(local_paths.RAW_DIR, 'horse_results_2022.csv'), index_col=0,  sep="\t")
horse_results.columns

C:\Users\SV8\AppData\Local\Temp\ipykernel_16016\1232359811.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  horse_results = pd.read_csv(os.path.join(local_paths.RAW_DIR, 'horse_results_2022.csv'), index_col=0,  sep="\t")


Index(['日付', '開催', '天気', 'R', 'レース名', '映像', '頭数', '枠番', '馬番', 'オッズ', '人気',
       '着順', '騎手', '斤量', '距離', '馬場', '馬場指数', 'タイム', '着差', 'ﾀｲﾑ指数', '通過', 'ペース',
       '上り', '馬体重', '厩舎ｺﾒﾝﾄ', '備考', '勝ち馬(2着馬)', '賞金', '受賞歴'],
      dtype='object')

In [5]:
horse_results['タイム'].iloc[1].split(':')

['1', '01.9']

In [11]:
df = horse_results.copy()

In [14]:
import numpy as np
time_list = []
for time in df['タイム']: 
    # NaNのチェック
    if pd.isna(time):
        print("NaN found")
        time_list.append(np.nan)
        continue
    times = str(time).split(':')
    total_seconds = int(times[0]) * 60 + float(times[1])
    time_list.append(total_seconds)
df['time'] = time_list



NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found
NaN found


In [18]:
df['time'].isnull().sum()

3492

In [68]:
horse_results['ペース']

horse_id
2019103610     0.0-37.6
2019103610     0.0-37.3
2019103610     0.0-37.6
2019103610     0.0-36.5
2019103610     0.0-37.4
                ...    
2018105939    30.6-36.5
2018105939    36.9-38.2
2018105939    35.8-40.0
2018105939    38.6-38.8
2018105939    38.3-38.1
Name: ペース, Length: 253546, dtype: object

In [19]:
horse_results['上り']

horse_id
2019103610    37.3
2019103610    37.2
2019103610    37.9
2019103610    37.3
2019103610    36.9
              ... 
2018105939    37.4
2018105939    37.6
2018105939    39.7
2018105939    38.2
2018105939    37.8
Name: 上り, Length: 253546, dtype: float64

## レース情報テーブル追加

In [4]:
%autoreload

In [15]:
results_p = pd.read_csv(os.path.join(local_paths.PREPROCESSED_DIR, 'results.csv'), index_col=0,  sep="\t")

In [16]:
results_p

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,frame,number,sex,age,impost,win,popularity,weight,weight_diff
27404,202301010101,2021100648,2021100648,2021100648,2021100648,5.0,1,1,1,2,55.0,23.9,5,434,-10
27405,202301010101,2021100159,2021100159,2021100159,2021100159,6.0,2,2,0,2,55.0,61.8,7,454,-6
27406,202301010101,2021100265,2021100265,2021100265,2021100265,7.0,3,3,1,2,53.0,18.8,4,404,-2
27403,202301010101,2021105553,2021105553,2021105553,2021105553,4.0,4,4,1,2,55.0,16.6,3,450,2
27400,202301010101,2021101429,2021101429,2021101429,2021101429,1.0,5,5,0,2,55.0,1.2,1,452,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33202,202310030812,2019102542,2019102542,2019102542,2019102542,1.0,6,8,1,4,53.0,3.5,1,442,-2
33207,202310030812,2020103754,2020103754,2020103754,2020103754,6.0,7,9,0,3,55.0,15.2,8,468,0
33205,202310030812,2020100400,2020100400,2020100400,2020100400,4.0,7,10,1,3,50.0,23.1,9,458,6
33203,202310030812,2020103663,2020103663,2020103663,2020103663,2.0,8,11,2,3,55.0,5.0,3,486,-6


In [17]:
horse_results_path = os.path.join(local_paths.PREPROCESSED_DIR, "horse_results.csv")
horse_results_p = pd.read_csv(horse_results_path, index_col=0, encoding='utf8', sep='\t')

In [18]:
horse_results_p

,horse_id,date,weather,race_class,n_horses,rank,race_type,course_len,ground_state,rank_diff,prize
0,2020103575,2024-08-31,3.0,5.0,15.0,15.0,1,1400,3,4.6,0.0
1,2020103575,2024-07-27,1.0,5.0,15.0,8.0,1,1200,0,0.9,0.0
2,2020103575,2024-05-19,0.0,5.0,15.0,4.0,1,1200,0,0.6,230.0
3,2020103575,2024-05-05,0.0,2.0,13.0,1.0,1,1200,0,0.0,800.0
4,2020103575,2024-02-25,3.0,2.0,15.0,7.0,1,1200,3,0.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...
189737,2019104657,2022-05-01,3.0,1.0,12.0,6.0,1,1800,3,1.3,0.0
189738,2019104657,2022-04-09,0.0,1.0,16.0,7.0,1,1800,0,2.6,0.0
189739,2019104657,2022-03-20,0.0,1.0,15.0,4.0,1,1800,2,2.1,78.0
189740,2019104657,2022-02-12,0.0,1.0,13.0,13.0,0,1800,0,3.0,0.0


### レース結果に日付がない → レース情報を取得・結合

In [19]:
%autoreload

In [21]:
race_id_list = results_p['race_id']
html_paths_race = prepare_html.get_html_race(race_id_list)

  4%|▍         | 1803/47273 [00:00<00:02, 17771.72it/s]

skipped: 202301010101
skipped: 202301010101
skipped: 202301010101
skipped: 202301010101
skipped: 202301010101
skipped: 202301010101
skipped: 202301010101
skipped: 202301010101
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010102
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010103
skipped: 202301010104
skipped: 202301010104
skipped: 202301010104
skipped: 202301010104
skipped: 202301010104
skipped: 202301010104
skipped: 202301010104
skipped: 202301010104
skipped: 202301010104
skipped: 202301010104
skipped: 202301010104
skipped: 2

 12%|█▏        | 5541/47273 [00:00<00:02, 15982.59it/s]

skipped: 202302020509
skipped: 202302020509
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020510
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020511
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020512
skipped: 202302020601
skipped: 202302020601
skipped: 2

 20%|█▉        | 9235/47273 [00:00<00:02, 16990.34it/s]

...

 88%|████████▊ | 41821/47273 [00:02<00:00, 17920.17it/s]

skipped: 202309030204
skipped: 202309030204
skipped: 202309030204
skipped: 202309030204
skipped: 202309030205
skipped: 202309030205
skipped: 202309030205
skipped: 202309030205
skipped: 202309030205
skipped: 202309030205
skipped: 202309030205
skipped: 202309030205
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030206
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030207
skipped: 202309030208
skipped: 2

100%|██████████| 47273/47273 [00:02<00:00, 17870.10it/s]

skipped: 202310010109
skipped: 202310010109
skipped: 202310010109
skipped: 202310010109
skipped: 202310010109
skipped: 202310010109
skipped: 202310010109
skipped: 202310010109
skipped: 202310010109
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010110
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010111
skipped: 202310010112
skipped: 202310010112
skipped: 202310010112
skipped: 202310010112
skipped: 2

['c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010102.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010102.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010102.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010102.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010102.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010102.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\

In [22]:
html_paths_race[:5]

['c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202301010101.bin']

In [26]:
with open(html_paths_race[0], 'rb') as f:
  html = f.read()

In [27]:
soup = BeautifulSoup(html, "lxml")

In [30]:
soup_info = soup.find('div', class_='data_intro')
soup_info

<div class="data_intro">
<dl class="racedata fc">
<dt>
1 R
</dt>
<dd>
<h1>2歳未勝利<!--img src="style/netkeiba.ja/image/race_grade_g2_01.png" alt=""--></h1>
<p>
<diary_snap_cut>
<span>芝右1200m / 天候 : 晴 / 芝 : 良 / 発走 : 09:50</span><br/>
<diary_snap_cut>
</diary_snap_cut>
</diary_snap_cut></p>
</dd>
</dl>
<p class="smalltxt">2023年7月22日 1回札幌1日目 2歳未勝利  (混)[指](馬齢)</p>
<diary_snap_cut>
<ul class="btn_link_list fc">
<li class="race_btn_MOVIE end">
<a href="/race/movie/202301010101"><img alt="レース映像" class="imgover" src="https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/race_btn_movie_on.png"/></a>
</li>
</ul>
</diary_snap_cut>
</div>

In [32]:
soup_info.find('h1').text

'2歳未勝利'

In [47]:
soup_state1 = soup_info.find('p').text.replace(' ', '')
soup_state1

'\n\n芝右1200m\xa0/\xa0天候:晴\xa0/\xa0芝:良\xa0/\xa0発走:09:50\n\n\n'

In [46]:
re.findall(r'[\w+:]+', soup_state1)

['芝右1200m', '天候:晴', '芝:良', '発走:09:50']

In [45]:
soup_state2 = soup_info.find_all('p')[1].text
soup_state2

'2023年7月22日 1回札幌1日目 2歳未勝利\xa0\xa0(混)[指](馬齢)'

In [50]:
re.findall(r'\w+', soup_state2)

['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '馬齢']

In [52]:
info_dict = {}
info_dict['title'] = soup_info.find('h1').text
info_dict['info1'] = re.findall(r'[\w+:]+', soup_info.find('p').text.replace(' ', ''))
info_dict['info2'] = re.findall(r'\w+', soup_info.find_all('p')[1].text)
info_dict

{'title': '2歳未勝利',
 'info1': ['芝右1200m', '天候:晴', '芝:良', '発走:09:50'],
 'info2': ['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '馬齢']}

In [56]:
pd.DataFrame().from_dict(info_dict, orient='index').T

,title,info1,info2
0,2歳未勝利,"[芝右1200m, 天候:晴, 芝:良, 発走:09:50]","[2023年7月22日, 1回札幌1日目, 2歳未勝利, 混, 指, 馬齢]"


In [3]:
race_info_path = os.path.join(local_paths.RAW_DIR, "race_info.csv")
race_info = pd.read_csv(race_info_path, encoding='utf8', sep='\t')

In [15]:
race_info

,race_id,title,info1,info2
0,202301010101,2歳未勝利,"['芝右1200m', '天候:晴', '芝:良', '発走:09:50']","['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '..."
1,202301010102,3歳未勝利,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
2,202301010103,3歳未勝利,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
3,202301010104,3歳未勝利,"['芝右1500m', '天候:晴', '芝:良', '発走:11:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '牝', '指', '..."
4,202301010105,2歳新馬,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']","['2023年7月22日', '1回札幌1日目', '2歳新馬', '混', '指', '馬齢']"
...,...,...,...,...
3451,202310030808,3歳以上1勝クラス,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:13:55']","['2023年9月3日', '3回小倉8日目', '3歳以上1勝クラス', '指', '定量']"
3452,202310030809,西日本新聞杯(2勝),"['芝右1200m', '天候:晴', '芝:良', '発走:14:25', '過去の西日本...","['2023年9月3日', '3回小倉8日目', '3歳以上2勝クラス', '混', '指'..."
3453,202310030810,宮崎ステークス(3勝),"['ダ右1700m', '天候:晴', 'ダート:良', '発走:15:01', '過去の宮...","['2023年9月3日', '3回小倉8日目', '3歳以上3勝クラス', '混', '特指..."
3454,202310030811,第43回小倉2歳ステークス(GIII),"['芝右1200m', '天候:晴', '芝:良', '発走:15:35', '過去の小倉2...","['2023年9月3日', '3回小倉8日目', '2歳オープン', '国際', '特指',..."


In [11]:
race_info['info1'][1]

"['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']"

In [237]:
info1_list = []
info2_list = []
for n in range(len(race_info)):
  info1 = race_info['info1'][n].replace("'", '').replace("[", '').replace("]", '').split(',')
  info2 = race_info['info2'][n].replace("'", '').replace("[", '').replace("]", '').split(',')
  info1_list.append(info1)
  info2_list.append(info2)

In [242]:
info2_df = pd.DataFrame({'info2': info2_list})

In [287]:
info2_df['info2']

0           [2023年7月22日,  1回札幌1日目,  2歳未勝利,  混,  指,  馬齢]
1               [2023年7月22日,  1回札幌1日目,  3歳未勝利,  指,  馬齢]
2               [2023年7月22日,  1回札幌1日目,  3歳未勝利,  指,  馬齢]
3           [2023年7月22日,  1回札幌1日目,  3歳未勝利,  牝,  指,  馬齢]
4            [2023年7月22日,  1回札幌1日目,  2歳新馬,  混,  指,  馬齢]
                             ...                       
3451         [2023年9月3日,  3回小倉8日目,  3歳以上1勝クラス,  指,  定量]
3452     [2023年9月3日,  3回小倉8日目,  3歳以上2勝クラス,  混,  指,  定量]
3453    [2023年9月3日,  3回小倉8日目,  3歳以上3勝クラス,  混,  特指,  定量]
3454      [2023年9月3日,  3回小倉8日目,  2歳オープン,  国際,  特指,  馬齢]
3455    [2023年9月3日,  3回小倉8日目,  3歳以上1勝クラス,  混,  特指,  定量]
Name: info2, Length: 3456, dtype: object

In [295]:
%autoreload

In [25]:
for info1 in info1_list[:3]:
  print(info1)

['芝右1200m', ' 天候:晴', ' 芝:良', ' 発走:09:50']
['ダ右1000m', ' 天候:晴', ' ダート:良', ' 発走:10:25']
['ダ右1700m', ' 天候:晴', ' ダート:良', ' 発走:10:55']


In [122]:
df = pd.DataFrame()

In [68]:
re.findall(r'\d+', info1_list[0][0])

['1200']

In [62]:
df

""


In [162]:
race_info['info1'][:20]

0                ['芝右1200m', '天候:晴', '芝:良', '発走:09:50']
1              ['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']
2              ['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']
3                ['芝右1500m', '天候:晴', '芝:良', '発走:11:25']
4              ['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']
5                ['芝右1800m', '天候:晴', '芝:良', '発走:12:45']
6                ['芝右1200m', '天候:晴', '芝:良', '発走:13:15']
7              ['ダ右1700m', '天候:晴', 'ダート:良', '発走:13:45']
8     ['芝右1800m', '天候:晴', '芝:良', '発走:14:15', '過去の北辰特別']
9     ['芝右2000m', '天候:晴', '芝:良', '発走:14:50', '過去のライラ...
10    ['芝右1200m', '天候:晴', '芝:良', '発走:15:25', '過去のTVh賞']
11             ['ダ右1700m', '天候:晴', 'ダート:良', '発走:16:05']
12             ['ダ右1000m', '天候:晴', 'ダート:良', '発走:09:50']
13             ['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:25']
14               ['芝右1200m', '天候:晴', '芝:良', '発走:10:55']
15             ['ダ右1700m', '天候:晴', 'ダート:良', '発走:11:25']
16               ['芝右1800m', '天候:晴', '芝:良', '発走:12:15']
17               ['芝右2000m', '天候:晴', '芝:良', '発走:

In [ ]:
race_info

In [209]:
df_p = pd.DataFrame()
info1_df = pd.DataFrame({'info1': info1_list})

In [310]:
df_p['place'] = info2_df['info2'].apply(lambda x: x[1])

In [317]:
df_p['place'].str.extract(r'\d回(.*?)\d日目')

,0
0,札幌
1,札幌
2,札幌
3,札幌
4,札幌
...,...
3451,小倉
3452,小倉
3453,小倉
3454,小倉


In [147]:
info1_df['info1'][0][1]

' 天候:晴'

In [210]:
df_p['weather'] = info1_df['info1'].apply(lambda x: x[1].split(':'))
df_p['weather'] = df_p['weather'].apply(lambda x: x[1] if len(x) > 1 else None)

In [223]:
df_p['ground_state'] = info1_df['info1'].apply(lambda x: x[2].split(':'))
df_p['ground_state'] = df_p['ground_state'].apply(lambda x: None if x[1] in ['晴', '曇', '雨', '小雨'] else x[1])

In [224]:
df_p.iloc[550:555]

,weather,ground_state
550,曇,稍重
551,曇,良
552,None,None
553,晴,良
554,晴,良


In [213]:
df_p['ground_state'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3456 entries, 0 to 3455
Series name: ground_state
Non-Null Count  Dtype 
--------------  ----- 
3456 non-null   object
dtypes: object(1)
memory usage: 27.1+ KB


In [226]:
race_info['title']

0                     2歳未勝利
1                     3歳未勝利
2                     3歳未勝利
3                     3歳未勝利
4                      2歳新馬
               ...         
3451              3歳以上1勝クラス
3452             西日本新聞杯(2勝)
3453            宮崎ステークス(3勝)
3454    第43回小倉2歳ステークス(GIII)
3455              3歳以上1勝クラス
Name: title, Length: 3456, dtype: object

In [230]:
df_p

,weather,ground_state,race_class
0,晴,良,1.0
1,晴,良,1.0
2,晴,良,1.0
3,晴,良,1.0
4,晴,良,0.0
...,...,...,...
3451,晴,良,2.0
3452,晴,良,NaN
3453,晴,良,NaN
3454,晴,良,NaN


In [ ]:
pd.to_datetime(info2_df['info2'][0][0], format='%Y年%m月%d日')

In [246]:
df_p['date'] = info2_df['info2'].apply(lambda x : x[0])
df_p['date'] = pd.to_datetime(df_p['date'], format='%Y年%m月%d日')

In [252]:
df_p = pd.DataFrame()

In [259]:
course_len_list = []
for info1 in info1_list:
  course_len = re.findall(r'\d+', info1[0])
  if course_len and int(course_len[0]) > 500:
    course_len_list.append(course_len[0])
  else:
    course_len_list.append(0)
df_p['course_len'] = course_len_list

In [260]:
df_p['course_len'].value_counts()

course_len
1800    799
1200    631
1400    440
1600    425
2000    368
1700    205
1000     86
2400     77
2200     70
1900     44
2600     39
2100     38
1150     37
0        28
1300     24
2860     21
1500     21
2500     15
3000     14
2750     12
2880     10
2970     10
3110      8
2770      6
3390      5
2910      3
3380      3
2300      2
3200      2
3140      2
3400      1
3210      1
3350      1
3330      1
3300      1
4100      1
3570      1
4250      1
3930      1
3170      1
3900      1
Name: count, dtype: int64

,race_id,date,race_type,around
0,202301010101,2023-07-22,0,0.0
1,202301010102,2023-07-22,1,0.0
2,202301010103,2023-07-22,1,0.0
3,202301010104,2023-07-22,0,0.0
4,202301010105,2023-07-22,1,0.0
...,...,...,...,...
3451,202310030808,2023-09-03,1,0.0
3452,202310030809,2023-09-03,0,0.0
3453,202310030810,2023-09-03,1,0.0
3454,202310030811,2023-09-03,0,0.0


In [277]:
race_info

,race_id,title,info1,info2
0,202301010101,2歳未勝利,"['芝右1200m', '天候:晴', '芝:良', '発走:09:50']","['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '..."
1,202301010102,3歳未勝利,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
2,202301010103,3歳未勝利,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
3,202301010104,3歳未勝利,"['芝右1500m', '天候:晴', '芝:良', '発走:11:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '牝', '指', '..."
4,202301010105,2歳新馬,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']","['2023年7月22日', '1回札幌1日目', '2歳新馬', '混', '指', '馬齢']"
...,...,...,...,...
3451,202310030808,3歳以上1勝クラス,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:13:55']","['2023年9月3日', '3回小倉8日目', '3歳以上1勝クラス', '指', '定量']"
3452,202310030809,西日本新聞杯(2勝),"['芝右1200m', '天候:晴', '芝:良', '発走:14:25', '過去の西日本...","['2023年9月3日', '3回小倉8日目', '3歳以上2勝クラス', '混', '指'..."
3453,202310030810,宮崎ステークス(3勝),"['ダ右1700m', '天候:晴', 'ダート:良', '発走:15:01', '過去の宮...","['2023年9月3日', '3回小倉8日目', '3歳以上3勝クラス', '混', '特指..."
3454,202310030811,第43回小倉2歳ステークス(GIII),"['芝右1200m', '天候:晴', '芝:良', '発走:15:35', '過去の小倉2...","['2023年9月3日', '3回小倉8日目', '2歳オープン', '国際', '特指',..."


In [268]:
len(course_len_list)

3456

In [269]:
df_p['course_len']

0       1200
1       1000
2       1700
3       1500
4       1700
        ... 
3451    1000
3452    1200
3453    1700
3454    1200
3455    2600
Name: course_len, Length: 3456, dtype: object

In [270]:
df_p

,race_id,date,race_type,around,course_len
0,202301010101,2023-07-22,0,1.0,1200
1,202301010102,2023-07-22,1,1.0,1000
2,202301010103,2023-07-22,1,1.0,1700
3,202301010104,2023-07-22,0,1.0,1500
4,202301010105,2023-07-22,1,1.0,1700
...,...,...,...,...,...
3451,202310030808,2023-09-03,1,1.0,1000
3452,202310030809,2023-09-03,0,1.0,1200
3453,202310030810,2023-09-03,1,1.0,1700
3454,202310030811,2023-09-03,0,1.0,1200


## 馬の過去成績データの特徴量作成
resultsとrace_infoを結合(results_add_infoとする) → results_add_infoの日付より前のデータで特徴量を作成する。
特徴量は、
- 戦績は過去５回の平均値, 最大値, 最小値、頭数は平均, 標準偏差、賞金は平均, 合計を算出し特徴量に含める
- race_typeなどのカテゴリ変数はダミー変数化して平均する

In [61]:
horse_results_path = os.path.join(local_paths.PREPROCESSED_DIR, "horse_results_2022.csv")
horse_results_p = pd.read_csv(horse_results_path, index_col=0, encoding='utf8', sep='\t')
race_info_path = os.path.join(local_paths.PREPROCESSED_DIR, "race_info_2022.csv")
race_info_p = pd.read_csv(race_info_path, index_col=0, encoding='utf8', sep='\t')
results_path = os.path.join(local_paths.PREPROCESSED_DIR, "results_2022.csv")
results_p = pd.read_csv(results_path, index_col=0, encoding='utf8', sep='\t')

In [62]:
horse_results_p

,horse_id,date,weather,race_class,n_horses,rank,race_type,course_len,ground_state,rank_diff,prize
0,2019103610,2023-11-03,0.0,NaN,12.0,5.0,1,1000,1.0,0.9,3.8
1,2019103610,2023-10-17,0.0,5.0,10.0,6.0,1,1000,1.0,1.0,0.0
2,2019103610,2023-10-04,0.0,NaN,10.0,6.0,1,1000,0.0,0.9,0.0
3,2019103610,2023-09-19,0.0,5.0,12.0,8.0,1,1000,1.0,1.7,0.0
4,2019103610,2023-09-05,0.0,5.0,11.0,3.0,1,1000,3.0,0.3,11.6
...,...,...,...,...,...,...,...,...,...,...,...
253541,2018105939,2021-06-26,1.0,2.0,14.0,4.0,1,2100,0.0,1.3,110.0
253542,2018105939,2021-05-29,0.0,2.0,16.0,2.0,1,1800,1.0,0.1,290.0
253543,2018105939,2021-05-02,1.0,1.0,15.0,1.0,1,2000,1.0,0.0,510.0
253544,2018105939,2021-03-06,1.0,1.0,12.0,2.0,1,1800,1.0,0.0,200.0


In [63]:
race_info_p

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place
0,202206010101,2022-01-05,1,0.0,1200.0,0.0,0.0,1.0,5.0
1,202206010102,2022-01-05,1,0.0,1800.0,0.0,0.0,1.0,5.0
2,202206010103,2022-01-05,1,0.0,1800.0,0.0,0.0,1.0,5.0
3,202206010104,2022-01-05,1,0.0,1800.0,0.0,0.0,0.0,5.0
4,202206010105,2022-01-05,0,0.0,2000.0,0.0,0.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...
3451,202209060908,2022-12-28,1,0.0,1800.0,0.0,0.0,2.0,8.0
3452,202209060909,2022-12-28,1,0.0,1400.0,0.0,0.0,NaN,8.0
3453,202209060910,2022-12-28,0,0.0,2000.0,0.0,0.0,NaN,8.0
3454,202209060911,2022-12-28,1,0.0,1800.0,0.0,0.0,NaN,8.0


In [7]:
results_p

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,frame,number,sex,age,impost,win_odds,popularity,weight,weight_diff
27404,202301010101,2021100648,1095,1128,281008,5.0,1,1,1,2,55.0,23.9,5,434,-10
27405,202301010101,2021100159,1157,1186,170800,6.0,2,2,0,2,55.0,61.8,7,454,-6
27406,202301010101,2021100265,1197,1192,320803,7.0,3,3,1,2,53.0,18.8,4,404,-2
27403,202301010101,2021105553,5339,1160,680031,4.0,4,4,1,2,55.0,16.6,3,450,2
27400,202301010101,2021101429,1170,1026,425031,1.0,5,5,0,2,55.0,1.2,1,452,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33202,202310030812,2019102542,1208,1161,17803,1.0,6,8,1,4,53.0,3.5,1,442,-2
33207,202310030812,2020103754,1116,1104,651031,6.0,7,9,0,3,55.0,15.2,8,468,0
33205,202310030812,2020100400,1193,1092,180800,4.0,7,10,1,3,50.0,23.1,9,458,6
33203,202310030812,2020103663,732,1092,226800,2.0,8,11,2,3,55.0,5.0,3,486,-6


In [8]:
results_add_info = results_p.merge(race_info_p, on='race_id', how='left')

In [9]:
results_add_info

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,frame,number,sex,age,...,weight,weight_diff,date,race_type,around,course_len,weather,ground_state,race_class,place
0,202301010101,2021100648,1095,1128,281008,5.0,1,1,1,2,...,434,-10,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
1,202301010101,2021100159,1157,1186,170800,6.0,2,2,0,2,...,454,-6,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
2,202301010101,2021100265,1197,1192,320803,7.0,3,3,1,2,...,404,-2,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
3,202301010101,2021105553,5339,1160,680031,4.0,4,4,1,2,...,450,2,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
4,202301010101,2021101429,1170,1026,425031,1.0,5,5,0,2,...,452,-4,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47268,202310030812,2019102542,1208,1161,17803,1.0,6,8,1,4,...,442,-2,2023-09-03,0,0.0,2600.0,0.0,0.0,2.0,9.0
47269,202310030812,2020103754,1116,1104,651031,6.0,7,9,0,3,...,468,0,2023-09-03,0,0.0,2600.0,0.0,0.0,2.0,9.0
47270,202310030812,2020100400,1193,1092,180800,4.0,7,10,1,3,...,458,6,2023-09-03,0,0.0,2600.0,0.0,0.0,2.0,9.0
47271,202310030812,2020103663,732,1092,226800,2.0,8,11,2,3,...,486,-6,2023-09-03,0,0.0,2600.0,0.0,0.0,2.0,9.0


In [70]:
class Horse:
  def __init__(
    self, 
    input_dir=local_paths.PREPROCESSED_DIR,
    output_dir=local_paths.FEATURES_DIR,
    save_filename='horse_features.csv',
    race_info_filename='race_info.csv',
    results_filename='results.csv',
    horse_results_filename='horse_results.csv'
  ) -> pd.DataFrame:
      
    # ファイルのパスを作成
    race_info_path = os.path.join(input_dir, race_info_filename)
    results_path = os.path.join(input_dir, results_filename)
    horse_results_path = os.path.join(input_dir, horse_results_filename)

    # データの読み込み
    race_info = pd.read_csv(race_info_path, index_col=0, encoding='utf8', sep='\t')
    results = pd.read_csv(results_path, index_col=0, encoding='utf8', sep='\t')
    horse_results = pd.read_csv(horse_results_path, index_col=0, encoding='utf8', sep='\t')

    # race_infoとresultsの結合
    self.results_add_info = results.merge(race_info, on='race_id', how='left')
    self.horse_results = horse_results

    # 過去の結果データをフィルタリングし、特徴量を作成
    self.df_f = self.filter()
    self.df_c = self.create_features()

    # 結果をCSVに保存
    self.df_c.to_csv(os.path.join(output_dir, save_filename), sep="\t")

  def filter(self):
    """馬ごとの過去の結果データをフィルタリング"""
    filtered_results = []
    for _, row in tqdm(self.results_add_info[:100].iterrows(), desc="Filtering past results"):
      horse_id = row['horse_id']
      date = row['date']
      past_results = self.horse_results[(self.horse_results['horse_id'] == horse_id) &
                                        (self.horse_results['date'] < date)]
      if not past_results.empty:
        # 参照日（現在のレースの日付）を追加
        past_results = past_results.copy()
        past_results['reference_date'] = date
        filtered_results.append(past_results)
    
    return pd.concat(filtered_results, ignore_index=True) if filtered_results else pd.DataFrame()

  def create_features(self):
    """過去の結果データを基に特徴量を作成"""
    past_results = self.filter()

    # 数値データに基づく特徴量を作成
    features = past_results.groupby(['horse_id', 'reference_date']).agg({
      'rank': ['mean', 'min', 'max', 'std'],     # 順位の平均、最小、最大、標準偏差
      'n_horses': ['mean'],                      # 出走頭数の平均
      'rank_diff': ['mean', 'std'],              # 着差の平均、標準偏差
      'prize': ['mean', 'sum'],                  # 賞金の平均、合計
      'course_len': ['mean', 'min', 'max'],      # コース距離の平均、最小、最大
      'date': ['max']                            # 過去のレースの日付の最大値（最後のレースの日付）
    }).reset_index()

    # カラム名をフラット化
    features.columns = ['_'.join(col).strip() for col in features.columns.values]
    features = features.rename(columns={'horse_id_': 'horse_id', 'reference_date_': 'reference_date'})

    # カテゴリ変数は最頻値を利用して特徴量を作成
    for col in ['weather', 'race_type', 'ground_state']:
      mode_values = past_results.groupby(['horse_id', 'reference_date'])[col].agg(
        lambda x: x.mode()[0] if not x.mode().empty else None
      ).reset_index(name=f'{col}_mode')
      # 最頻値をマージ
      features = features.merge(mode_values, on=['horse_id', 'reference_date'], how='left')

    # 順位の一貫性を特徴量として追加（順位の最大値と最小値の差）
    features['consistency'] = features['rank_max'] - features['rank_min']

    # 最頻値のコース距離を計算
    course_len_mode = past_results.groupby(['horse_id', 'reference_date'])['course_len'].agg(
      lambda x: x.mode()[0] if not x.mode().empty else None
    ).reset_index(name='course_len_mode')

    # 同じコース距離での過去の平均順位を計算
    avg_rank_by_course_len = past_results.groupby(['horse_id', 'course_len']).agg({
      'rank': 'mean'
    }).reset_index().rename(columns={'rank': 'same_course_len_avg_rank'})

    # featuresに最頻値のコース距離での過去の平均順位をマージ
    features = features.merge(course_len_mode, on=['horse_id', 'reference_date'], how='left')
    features = features.merge(avg_rank_by_course_len, 
                              left_on=['horse_id', 'course_len_mode'], 
                              right_on=['horse_id', 'course_len'], 
                              how='left')

    # 最後のレースから次のレースまでの日数を計算
    features['last_race_date'] = pd.to_datetime(features['date_max'])  # 最後のレース日
    features['reference_date'] = pd.to_datetime(features['reference_date'])  # 現在のレース日
    features['days_since_last_race'] = (features['reference_date'] - features['last_race_date']).dt.days

    # 不要なカラムを削除
    features = features.drop(columns=['last_race_date', 'date_max'])

    return features



In [71]:
h = Horse(save_filename='horse_features_2022.csv',
                          race_info_filename='race_info_2022.csv',
                          results_filename='results_2022.csv', 
                          horse_results_filename='horse_results_2022.csv')

Filtering past results: 100it [00:01, 68.94it/s]
Filtering past results: 100it [00:01, 69.33it/s]


In [72]:
h.df_f

,horse_id,date,weather,race_class,n_horses,rank,race_type,course_len,ground_state,rank_diff,prize,reference_date
0,2020100678,2022-06-26,0.0,0.0,12.0,4.0,0,1800,0.0,0.7,110.0,2022-07-23
1,2020105743,2022-07-09,1.0,1.0,7.0,5.0,0,1200,0.0,1.2,52.0,2022-07-23
2,2020105743,2022-06-26,0.0,0.0,8.0,4.0,0,1200,0.0,0.4,110.0,2022-07-23
3,2020102526,2022-07-17,2.0,0.0,11.0,6.0,0,1800,2.0,1.2,0.0,2022-07-23
4,2020102360,2022-07-02,0.0,1.0,10.0,7.0,0,1200,0.0,0.8,0.0,2022-07-23
...,...,...,...,...,...,...,...,...,...,...,...,...
587,2019104079,2022-02-06,0.0,1.0,16.0,4.0,1,1700,0.0,0.3,78.0,2022-07-23
588,2019104079,2022-01-23,2.0,1.0,14.0,5.0,0,2000,2.0,1.4,52.0,2022-07-23
589,2019104079,2021-11-21,0.0,1.0,11.0,5.0,0,2000,0.0,0.5,51.0,2022-07-23
590,2019104079,2021-10-23,0.0,1.0,9.0,4.0,0,1800,0.0,1.0,77.0,2022-07-23


In [73]:
h.df_c

,horse_id,reference_date,rank_mean,rank_min,rank_max,rank_std,n_horses_mean,rank_diff_mean,rank_diff_std,prize_mean,...,course_len_min,course_len_max,weather_mode,race_type_mode,ground_state_mode,consistency,course_len_mode,course_len,same_course_len_avg_rank,days_since_last_race
0,2016101167,2022-07-23,5.484848,1.0,13.0,3.327172,14.212121,0.615152,0.423616,93.000000,...,1000,1800,0.0,1,0.0,12.0,1200,1200,5.450000,13
1,2017110112,2022-07-23,9.375000,1.0,17.0,4.529533,13.687500,1.362500,0.760592,94.187500,...,1000,1700,0.0,0,0.0,16.0,1400,1400,9.142857,14
2,2018100298,2022-07-23,8.000000,1.0,16.0,4.365641,14.833333,1.533333,1.183713,55.055556,...,1000,1400,0.0,1,0.0,15.0,1400,1400,8.750000,20
3,2018101782,2022-07-23,5.550000,1.0,15.0,3.886210,15.650000,0.465000,0.388350,115.610000,...,1000,1200,0.0,0,0.0,14.0,1200,1200,5.684211,34
4,2018102593,2022-07-23,5.846154,1.0,15.0,3.826359,14.307692,1.300000,1.243651,98.307692,...,1000,1800,0.0,1,0.0,14.0,1200,1200,4.833333,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,2020100678,2022-07-23,4.000000,4.0,4.0,NaN,12.000000,0.700000,NaN,110.000000,...,1800,1800,0.0,0,0.0,0.0,1800,1800,4.000000,27
82,2020102360,2022-07-23,5.500000,4.0,7.0,2.121320,9.500000,1.450000,0.919239,55.000000,...,1200,1200,0.0,0,0.0,3.0,1200,1200,5.500000,21
83,2020102526,2022-07-23,6.000000,6.0,6.0,NaN,11.000000,1.200000,NaN,0.000000,...,1800,1800,2.0,0,2.0,0.0,1800,1800,6.000000,6
84,2020102535,2022-07-23,3.000000,3.0,3.0,NaN,6.000000,0.800000,NaN,180.000000,...,1800,1800,0.0,0,0.0,0.0,1800,1800,3.000000,13


In [14]:
h.df_c['weather_mode']

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
     ... 
81    0.0
82    0.0
83    2.0
84    0.0
85    0.0
Name: weather_mode, Length: 86, dtype: float64

In [15]:
h.df_c[h.df_c['horse_id'].isin([2021106812])]

,horse_id,reference_date,rank_mean,rank_min,rank_max,rank_std,n_horses_mean,rank_diff_mean,rank_diff_std,prize_mean,prize_sum,course_len_mean,course_len_min,course_len_max,weather_mode,race_type_mode,ground_state_mode,consistency,days_since_last_race


In [82]:
h.df_f[(h.df_f['horse_id'].isin([2017110112])) & (h.df_f['course_len']==1400)]['rank'].sum() / len(h.df_f[(h.df_f['horse_id'].isin([2017110112])) & (h.df_f['course_len']==1400)])

np.float64(9.142857142857142)

In [80]:
h.df_f[(h.df_f['horse_id'].isin([2017110112])) & (h.df_f['course_len']==1400)]

,horse_id,date,weather,race_class,n_horses,rank,race_type,course_len,ground_state,rank_diff,prize,reference_date
389,2017110112,2022-04-28,0.0,5.0,12.0,7.0,1,1400,1.0,0.6,0.0,2022-07-23
390,2017110112,2022-03-21,0.0,2.0,16.0,8.0,1,1400,1.0,1.4,0.0,2022-07-23
391,2017110112,2022-01-10,0.0,2.0,15.0,10.0,1,1400,0.0,1.4,0.0,2022-07-23
393,2017110112,2021-06-13,1.0,2.0,10.0,10.0,0,1400,1.0,2.4,0.0,2022-07-23
394,2017110112,2020-11-23,0.0,2.0,18.0,17.0,0,1400,0.0,1.2,0.0,2022-07-23
398,2017110112,2020-03-07,0.0,2.0,10.0,10.0,0,1400,0.0,2.6,0.0,2022-07-23
400,2017110112,2020-01-18,1.0,NaN,10.0,2.0,0,1400,0.0,0.2,807.0,2022-07-23


In [84]:
h.df_f[h.df_f['horse_id']==2018100298]

,horse_id,date,weather,race_class,n_horses,rank,race_type,course_len,ground_state,rank_diff,prize,reference_date
288,2018100298,2022-07-03,0.0,2.0,11.0,6.0,1,1000,1.0,1.3,0.0,2022-07-23
289,2018100298,2022-04-10,0.0,2.0,16.0,9.0,1,1200,0.0,1.0,0.0,2022-07-23
290,2018100298,2022-03-13,0.0,2.0,16.0,9.0,1,1200,0.0,1.1,0.0,2022-07-23
291,2018100298,2022-01-23,1.0,2.0,16.0,12.0,1,1200,0.0,1.2,0.0,2022-07-23
292,2018100298,2021-12-12,0.0,2.0,16.0,8.0,1,1200,0.0,1.4,0.0,2022-07-23
293,2018100298,2021-11-20,0.0,2.0,16.0,7.0,1,1150,0.0,0.7,0.0,2022-07-23
294,2018100298,2021-10-16,3.0,2.0,16.0,16.0,1,1400,0.0,3.9,0.0,2022-07-23
295,2018100298,2021-09-04,0.0,2.0,12.0,5.0,1,1000,0.0,0.4,76.0,2022-07-23
296,2018100298,2021-07-31,1.0,1.0,12.0,1.0,1,1000,0.0,0.0,510.0,2022-07-23
297,2018100298,2021-06-26,0.0,1.0,16.0,5.0,1,1300,1.0,0.7,51.0,2022-07-23


In [69]:
results_add_info

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,frame,number,sex,age,...,weight,weight_diff,date,race_type,around,course_len,weather,ground_state,race_class,place
0,202301010101,2021100648,1095,1128,281008,5.0,1,1,1,2,...,434,-10,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
1,202301010101,2021100159,1157,1186,170800,6.0,2,2,0,2,...,454,-6,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
2,202301010101,2021100265,1197,1192,320803,7.0,3,3,1,2,...,404,-2,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
3,202301010101,2021105553,5339,1160,680031,4.0,4,4,1,2,...,450,2,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
4,202301010101,2021101429,1170,1026,425031,1.0,5,5,0,2,...,452,-4,2023-07-22,0,0.0,1200.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47268,202310030812,2019102542,1208,1161,17803,1.0,6,8,1,4,...,442,-2,2023-09-03,0,0.0,2600.0,0.0,0.0,2.0,9.0
47269,202310030812,2020103754,1116,1104,651031,6.0,7,9,0,3,...,468,0,2023-09-03,0,0.0,2600.0,0.0,0.0,2.0,9.0
47270,202310030812,2020100400,1193,1092,180800,4.0,7,10,1,3,...,458,6,2023-09-03,0,0.0,2600.0,0.0,0.0,2.0,9.0
47271,202310030812,2020103663,732,1092,226800,2.0,8,11,2,3,...,486,-6,2023-09-03,0,0.0,2600.0,0.0,0.0,2.0,9.0


In [492]:
horse_results[horse_results['horse_id'].isin([2021105553])]

,horse_id,rank,n_horses,prize,date
161315,2021105553,5.0,16.0,55.0,2024-08-31
161316,2021105553,5.0,16.0,55.0,2024-08-17
161317,2021105553,10.0,15.0,0.0,2024-05-19
161318,2021105553,4.0,16.0,83.0,2024-05-04
161319,2021105553,5.0,16.0,55.0,2024-04-06
161320,2021105553,7.0,14.0,0.0,2024-03-03
161321,2021105553,6.0,16.0,0.0,2024-02-11
161322,2021105553,5.0,11.0,55.0,2024-01-27
161323,2021105553,3.0,16.0,140.0,2023-11-19
161324,2021105553,3.0,15.0,140.0,2023-11-05


In [402]:
horse_id

2021101429

In [325]:
agg_features = horse_results_p.groupby('horse_id').agg({
    'rank': ['mean', 'std', 'min', 'max'],
    'n_horses': ['mean', 'std'],
    'prize': ['mean', 'sum']
}).reset_index()

In [326]:
agg_features

horse_id      rank                        n_horses            \
                       mean       std  min   max       mean       std   
0      2011106610  7.585366  3.463928  1.0  14.0  11.658537  2.480639   
1      2012100683  5.111111  3.316861  1.0  15.0  12.555556  2.599506   
2      2012103532  4.833333  2.933013  1.0  14.0  13.121212  2.471469   
3      2012104463  6.206897  4.836842  1.0  18.0  13.517241  2.590410   
4      2012104504  6.421053  4.765753  1.0  16.0  13.473684  2.036324   
...           ...       ...       ...  ...   ...        ...       ...   
11680  2021110155  4.250000  1.908627  2.0   7.0  14.000000  2.878492   
11681  2021110156  3.714286  1.704336  2.0   7.0  14.571429  1.988060   
11682  2021110157  2.500000  2.121320  1.0   4.0  10.500000  0.707107   
11683  2021110160  3.600000  4.219005  1.0  11.0  15.800000  2.588436   
11684  2021110161  5.142857  4.140393  1.0  12.0  12.857143  3.760699   

            prize           
             mean      sum  
0       87.378049   7165.0  
1      282.041667  20307.0  
2      312.913636  20652.3  
3      397.603448  23061.0  
4      325.254386  18539.5  
...           ...      ...  
11680   67.675000    541.4  
11681  137.857143    965.0  
11682  440.000000    880.0  
11683  589.340000   2946.7  
11684  257.142857   1800.0  

[11685 rows x 9 columns]

In [327]:
agg_features.columns.values

array([('horse_id', ''), ('rank', 'mean'), ('rank', 'std'),
       ('rank', 'min'), ('rank', 'max'), ('n_horses', 'mean'),
       ('n_horses', 'std'), ('prize', 'mean'), ('prize', 'sum')],
      dtype=object)

In [351]:
agg_features.columns = ['_'.join(col).strip() for col in agg_features.columns.values]
agg_features = agg_features.rename(columns={'horse_id_': 'horse_id'})

In [352]:
agg_features

,horse_id,rank_mean,rank_std,rank_min,rank_max,n_horses_mean,n_horses_std,prize_mean,prize_sum
0,2011106610,7.585366,3.463928,1.0,14.0,11.658537,2.480639,87.378049,7165.0
1,2012100683,5.111111,3.316861,1.0,15.0,12.555556,2.599506,282.041667,20307.0
2,2012103532,4.833333,2.933013,1.0,14.0,13.121212,2.471469,312.913636,20652.3
3,2012104463,6.206897,4.836842,1.0,18.0,13.517241,2.590410,397.603448,23061.0
4,2012104504,6.421053,4.765753,1.0,16.0,13.473684,2.036324,325.254386,18539.5
...,...,...,...,...,...,...,...,...,...
11680,2021110155,4.250000,1.908627,2.0,7.0,14.000000,2.878492,67.675000,541.4
11681,2021110156,3.714286,1.704336,2.0,7.0,14.571429,1.988060,137.857143,965.0
11682,2021110157,2.500000,2.121320,1.0,4.0,10.500000,0.707107,440.000000,880.0
11683,2021110160,3.600000,4.219005,1.0,11.0,15.800000,2.588436,589.340000,2946.7


In [334]:
encoded_features = pd.get_dummies(horse_results_p[['horse_id', 'race_class', 'race_type', 'weather']], 
                                  columns=['race_class', 'race_type', 'weather'])

In [335]:
encoded_features

,horse_id,race_class_0.0,race_class_1.0,race_class_2.0,race_class_3.0,race_class_4.0,race_class_5.0,race_class_7.0,race_class_8.0,race_type_0,race_type_1,race_type_2,weather_0.0,weather_1.0,weather_2.0,weather_3.0,weather_4.0,weather_5.0
0,2020103575,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False
1,2020103575,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False
2,2020103575,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False
3,2020103575,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,False
4,2020103575,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189737,2019104657,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False
189738,2019104657,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False
189739,2019104657,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False
189740,2019104657,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False


In [348]:
# horse_idごとに集約（平均または合計）
encoded_features = encoded_features.groupby('horse_id').mean().reset_index()

In [349]:
encoded_features

,horse_id,race_class_0.0,race_class_1.0,race_class_2.0,race_class_3.0,race_class_4.0,race_class_5.0,race_class_7.0,race_class_8.0,race_type_0,race_type_1,race_type_2,weather_0.0,weather_1.0,weather_2.0,weather_3.0,weather_4.0,weather_5.0
0,2011106610,0.012195,0.048780,0.036585,0.219512,0.000000,0.573171,0.0,0.0,0.695122,0.109756,0.195122,0.670732,0.231707,0.060976,0.024390,0.0,0.012195
1,2012100683,0.013889,0.180556,0.069444,0.111111,0.069444,0.263889,0.0,0.0,0.152778,0.416667,0.430556,0.569444,0.291667,0.069444,0.069444,0.0,0.000000
2,2012103532,0.015152,0.181818,0.227273,0.030303,0.075758,0.333333,0.0,0.0,0.545455,0.000000,0.454545,0.651515,0.242424,0.090909,0.015152,0.0,0.000000
3,2012104463,0.017241,0.086207,0.017241,0.000000,0.155172,0.413793,0.0,0.0,0.637931,0.017241,0.344828,0.672414,0.241379,0.051724,0.034483,0.0,0.000000
4,2012104504,0.017544,0.070175,0.035088,0.035088,0.350877,0.385965,0.0,0.0,0.000000,0.789474,0.210526,0.666667,0.245614,0.052632,0.035088,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11680,2021110155,0.125000,0.750000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.125000,0.875000,0.000000,0.500000,0.375000,0.125000,0.000000,0.0,0.000000
11681,2021110156,0.142857,0.714286,0.142857,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.571429,0.285714,0.000000,0.142857,0.0,0.000000
11682,2021110157,0.500000,0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000
11683,2021110160,0.200000,0.000000,0.400000,0.000000,0.000000,0.200000,0.0,0.0,1.000000,0.000000,0.000000,0.800000,0.200000,0.000000,0.000000,0.0,0.000000


In [345]:
# 過去Xレースの成績
X = 5
horse_results_p['rank_diff'] = horse_results_p.groupby('horse_id')['rank'].diff()  # 順位の変動量

# 過去Xレースの平均順位
horse_results_p['rank_rolling_mean'] = horse_results_p.groupby('horse_id')['rank'].rolling(window=X, min_periods=1).mean().reset_index(0, drop=True)

# 良馬場の平均順位
good_state_rank = horse_results_p[horse_results_p['ground_state'].isin([0])].groupby('horse_id')['rank'].mean().reset_index()
good_state_rank = good_state_rank.rename(columns={'rank': 'good_state_rank_mean'})

In [346]:
good_state_rank

,horse_id,good_state_rank_mean
0,2011106610,8.049180
1,2012100683,5.400000
2,2012103532,4.604167
3,2012104463,6.387755
4,2012104504,5.820513
...,...,...
11615,2021110155,4.666667
11616,2021110156,4.000000
11617,2021110157,4.000000
11618,2021110160,1.750000


In [353]:
final_features = agg_features.merge(encoded_features, on='horse_id', how='left')
final_features = final_features.merge(good_state_rank, on='horse_id', how='left')

In [354]:
final_features

,horse_id,rank_mean,rank_std,rank_min,rank_max,n_horses_mean,n_horses_std,prize_mean,prize_sum,race_class_0.0,...,race_type_0,race_type_1,race_type_2,weather_0.0,weather_1.0,weather_2.0,weather_3.0,weather_4.0,weather_5.0,good_state_rank_mean
0,2011106610,7.585366,3.463928,1.0,14.0,11.658537,2.480639,87.378049,7165.0,0.012195,...,0.695122,0.109756,0.195122,0.670732,0.231707,0.060976,0.024390,0.0,0.012195,8.049180
1,2012100683,5.111111,3.316861,1.0,15.0,12.555556,2.599506,282.041667,20307.0,0.013889,...,0.152778,0.416667,0.430556,0.569444,0.291667,0.069444,0.069444,0.0,0.000000,5.400000
2,2012103532,4.833333,2.933013,1.0,14.0,13.121212,2.471469,312.913636,20652.3,0.015152,...,0.545455,0.000000,0.454545,0.651515,0.242424,0.090909,0.015152,0.0,0.000000,4.604167
3,2012104463,6.206897,4.836842,1.0,18.0,13.517241,2.590410,397.603448,23061.0,0.017241,...,0.637931,0.017241,0.344828,0.672414,0.241379,0.051724,0.034483,0.0,0.000000,6.387755
4,2012104504,6.421053,4.765753,1.0,16.0,13.473684,2.036324,325.254386,18539.5,0.017544,...,0.000000,0.789474,0.210526,0.666667,0.245614,0.052632,0.035088,0.0,0.000000,5.820513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11680,2021110155,4.250000,1.908627,2.0,7.0,14.000000,2.878492,67.675000,541.4,0.125000,...,0.125000,0.875000,0.000000,0.500000,0.375000,0.125000,0.000000,0.0,0.000000,4.666667
11681,2021110156,3.714286,1.704336,2.0,7.0,14.571429,1.988060,137.857143,965.0,0.142857,...,0.000000,1.000000,0.000000,0.571429,0.285714,0.000000,0.142857,0.0,0.000000,4.000000
11682,2021110157,2.500000,2.121320,1.0,4.0,10.500000,0.707107,440.000000,880.0,0.500000,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,4.000000
11683,2021110160,3.600000,4.219005,1.0,11.0,15.800000,2.588436,589.340000,2946.7,0.200000,...,1.000000,0.000000,0.000000,0.800000,0.200000,0.000000,0.000000,0.0,0.000000,1.750000


## 払い戻しデータの追加

In [2]:
race_id_list = pd.read_pickle(os.path.join(local_paths.LISTS_DIR, 'race_id_list_2022.pickle'))
html_paths_race = prepare_html.get_html_race(race_id_list)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\SV8\\m\\p\\keiba\\data\\lists\\race_id_list.pickle'

In [30]:
html_paths_race[:10]

['c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010101.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010102.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010103.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010104.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010105.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010106.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010107.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010108.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010109.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\race\\202306010110.bin']

In [42]:
dfs = {}
for html_path in tqdm(html_paths_race[:10]):
  with open(html_path, "rb") as f:
    try:
      html = f.read()
      soup = BeautifulSoup(html, "lxml").find_all('table', class_='pay_table_01')

      tr_list = soup[0].find_all('tr')
      for tr in soup[1].find_all('tr'):
        tr_list.append(tr)

      returns_dict={}
      for tr in tr_list:
        key = tr.find('th').text
        tds = tr.find_all('td')
        returns_dict[key] = []
        for n in re.findall(r'\d{1,3}(?:,\d{3})*', str(tds[0])):
          returns_dict[key].append(n)
        for n in re.findall(r'\d{1,3}(?:,\d{3})*', str(tds[1])):
          returns_dict[key].append(n)

      df = pd.DataFrame({key: [value] for key, value in returns_dict.items()})

      race_id = re.search(r'\d{12}', html_path).group()
      df.index = [race_id] * len(df)
      dfs[race_id] = df

    except IndexError as e:
      print(f"table not found at {race_id}")
      continue

concat_df = pd.concat(dfs.values())
concat_df.index.name = "race_id"
concat_df.columns = concat_df.columns.str.replace(' ', '')

100%|██████████| 10/10 [00:00<00:00, 17.61it/s]


In [43]:
concat_df

,単勝,複勝,枠連,馬連,ワイド,馬単,三連複,三連単
race_id,,,,,,,,
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"


In [44]:
dfs

{'202306010101':                      単勝                          複勝           枠連  \
 202306010110  [13, 350]  [13, 11, 2, 160, 310, 350]  [7, 8, 270]   
 
                            馬連                                      ワイド  \
 202306010110  [11, 13, 1,760]  [11, 13, 2, 13, 2, 11, 720, 710, 1,860]   
 
                            馬単                 三連複                  三連単  
 202306010110  [13, 11, 3,080]  [2, 11, 13, 6,240]  [13, 11, 2, 25,690]  ,
 '202306010102':                      単勝                          複勝           枠連  \
 202306010110  [13, 350]  [13, 11, 2, 160, 310, 350]  [7, 8, 270]   
 
                            馬連                                      ワイド  \
 202306010110  [11, 13, 1,760]  [11, 13, 2, 13, 2, 11, 720, 710, 1,860]   
 
                            馬単                 三連複                  三連単  
 202306010110  [13, 11, 3,080]  [2, 11, 13, 6,240]  [13, 11, 2, 25,690]  ,
 '202306010103':                      単勝                          複勝           枠連  \
 

In [5]:
soup[0]

<table cellpadding="0" cellspacing="1" class="pay_table_01" summary="払い戻し">
<tr>
<th class="tan">単勝</th>
<td>13</td>
<td class="txt_r">350</td>
<td class="txt_r">2</td>
</tr>
<tr>
<th align="center" class="fuku">複勝</th>
<td>13<br/>11<br/>2</td>
<td class="txt_r">160<br/>310<br/>350</td>
<td class="txt_r">2<br/>4<br/>5</td>
</tr>
<tr>
<th align="center" class="waku">枠連</th>
<td>7 - 8</td>
<td class="txt_r">270</td>
<td class="txt_r">1</td>
</tr>
<tr>
<th align="center" class="uren">馬連</th>
<td>11 - 13</td>
<td class="txt_r">1,760</td>
<td class="txt_r">5</td>
</tr>
</table>

In [6]:
tr_list = soup[0].find_all('tr')
tr_list

[<tr>
 <th class="tan">単勝</th>
 <td>13</td>
 <td class="txt_r">350</td>
 <td class="txt_r">2</td>
 </tr>,
 <tr>
 <th align="center" class="fuku">複勝</th>
 <td>13<br/>11<br/>2</td>
 <td class="txt_r">160<br/>310<br/>350</td>
 <td class="txt_r">2<br/>4<br/>5</td>
 </tr>,
 <tr>
 <th align="center" class="waku">枠連</th>
 <td>7 - 8</td>
 <td class="txt_r">270</td>
 <td class="txt_r">1</td>
 </tr>,
 <tr>
 <th align="center" class="uren">馬連</th>
 <td>11 - 13</td>
 <td class="txt_r">1,760</td>
 <td class="txt_r">5</td>
 </tr>]

In [7]:
for tr in soup[1].find_all('tr'):
  tr_list.append(tr)

In [8]:
tr_list

[<tr>
 <th class="tan">単勝</th>
 <td>13</td>
 <td class="txt_r">350</td>
 <td class="txt_r">2</td>
 </tr>,
 <tr>
 <th align="center" class="fuku">複勝</th>
 <td>13<br/>11<br/>2</td>
 <td class="txt_r">160<br/>310<br/>350</td>
 <td class="txt_r">2<br/>4<br/>5</td>
 </tr>,
 <tr>
 <th align="center" class="waku">枠連</th>
 <td>7 - 8</td>
 <td class="txt_r">270</td>
 <td class="txt_r">1</td>
 </tr>,
 <tr>
 <th align="center" class="uren">馬連</th>
 <td>11 - 13</td>
 <td class="txt_r">1,760</td>
 <td class="txt_r">5</td>
 </tr>,
 <tr>
 <th class="wide">ワイド</th>
 <td>11 - 13<br/>2 - 13<br/>2 - 11</td>
 <td class="txt_r">720<br/>710<br/>1,860</td>
 <td class="txt_r">6<br/>5<br/>21</td>
 </tr>,
 <tr>
 <th class="utan">馬単</th>
 <td>13 → 11</td>
 <td class="txt_r">3,080</td>
 <td class="txt_r">9</td>
 </tr>,
 <tr>
 <th class="sanfuku">三連複</th>
 <td>2 - 11 - 13</td>
 <td class="txt_r">6,240</td>
 <td class="txt_r">18</td>
 </tr>,
 <tr>
 <th class="santan">三連単</th>
 <td>13 → 11 → 2</td>
 <td class="txt_r

In [9]:
returns_dict={}
for tr in tr_list:
  key = tr.find('th').text
  tds = tr.find_all('td')
  returns_dict[key] = []
  for n in re.findall(r'\d{1,3}(?:,\d{3})*', str(tds[0])):
    returns_dict[key].append(n)
  for n in re.findall(r'\d{1,3}(?:,\d{3})*', str(tds[1])):
    returns_dict[key].append(n)

In [10]:
returns_dict

{'単勝': ['13', '350'],
 '複勝': ['13', '11', '2', '160', '310', '350'],
 '枠連': ['7', '8', '270'],
 '馬連': ['11', '13', '1,760'],
 'ワイド': ['11', '13', '2', '13', '2', '11', '720', '710', '1,860'],
 '馬単': ['13', '11', '3,080'],
 '三連複': ['2', '11', '13', '6,240'],
 '三連単': ['13', '11', '2', '25,690']}

In [11]:
tr_list[0].find('th')

<th class="tan">単勝</th>

In [204]:
for tr in tr_list[:2]:
  tds = tr.find_all('td')
  tds

In [209]:
re.findall(r'\d+', str(tds[0]))

['13', '11', '2']

In [15]:
df = pd.DataFrame({key: [value] for key, value in returns_dict.items()})

In [18]:
df

,単勝,複勝,枠連,馬連,ワイド,馬単,三連複,三連単
race_id,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"


In [21]:
df

,単勝,複勝,枠連,馬連,ワイド,馬単,三連複,三連単
202306010110,"[13, 350]","[13, 11, 2, 160, 310, 350]","[7, 8, 270]","[11, 13, 1,760]","[11, 13, 2, 13, 2, 11, 720, 710, 1,860]","[13, 11, 3,080]","[2, 11, 13, 6,240]","[13, 11, 2, 25,690]"


### 払い戻しの前処理

In [101]:
returns_22_path = os.path.join(local_paths.RAW_DIR, 'returns_2023.csv')
df_o = pd.read_csv(returns_22_path,index_col=0, sep="\t")

In [102]:
df_o

,単勝,複勝,枠連,馬連,ワイド,馬単,三連複,三連単
race_id,,,,,,,,
202306010101,"[14, 180]","[14, 5, 1, 110, 150, 480]","[3, 7, 470]","[5, 14, 470]","[5, 14, 1, 14, 1, 5, 220, 790, 1550]","[14, 5, 660]","[1, 5, 14, 3040]","[14, 5, 1, 8890]"
202306010102,"[8, 1380]","[8, 7, 13, 460, 440, 400]","[4, 4, 7010]","[7, 8, 6420]","[7, 8, 8, 13, 7, 13, 1930, 2810, 2490]","[8, 7, 15240]","[7, 8, 13, 32390]","[8, 7, 13, 199190]"
202306010103,"[16, 1590]","[16, 9, 2, 540, 1000, 440]","[5, 8, 2580]","[9, 16, 24290]","[9, 16, 2, 16, 2, 9, 6100, 3510, 5930]","[16, 9, 47840]","[2, 9, 16, 117380]","[16, 9, 2, 649440]"
202306010104,"[7, 1590]","[7, 13, 1, 330, 290, 150]","[4, 7, 660]","[7, 13, 6650]","[7, 13, 1, 7, 1, 13, 1710, 970, 830]","[7, 13, 13100]","[1, 7, 13, 7310]","[7, 13, 1, 78280]"
202306010105,"[1, 690]","[1, 3, 16, 180, 150, 170]","[1, 2, 1130]","[1, 3, 1220]","[1, 3, 1, 16, 3, 16, 510, 750, 410]","[1, 3, 2770]","[1, 3, 16, 1860]","[1, 3, 16, 13100]"
...,...,...,...,...,...,...,...,...
202309050908,"[8, 270]","[8, 14, 3, 160, 170, 650]","[5, 8, 520]","[8, 14, 840]","[8, 14, 3, 8, 3, 14, 440, 1960, 3500]","[8, 14, 1490]","[3, 8, 14, 12740]","[8, 14, 3, 35560]"
202309050909,"[16, 350]","[16, 9, 14, 200, 1410, 460]","[5, 8, 1970]","[9, 16, 12570]","[9, 16, 14, 16, 9, 14, 3580, 1190, 6950]","[16, 9, 15500]","[9, 14, 16, 46670]","[16, 9, 14, 144610]"
202309050910,"[11, 1710]","[11, 6, 9, 390, 200, 190]","[6, 8, 630]","[6, 11, 3770]","[6, 11, 9, 11, 6, 9, 1160, 1600, 630]","[11, 6, 8960]","[6, 9, 11, 7020]","[11, 6, 9, 68800]"


In [103]:
df_o.columns

Index(['単勝', '複勝', '枠連', '馬連', 'ワイド', '馬単', '三連複', '三連単'], dtype='object')

In [104]:
df_o.iloc[1, 1]

'[8, 7, 13, 460, 440, 400]'

In [105]:
import pandas as pd

df = df_o.copy()

# 結果を格納するためのデータフレーム
df_a = pd.DataFrame()

# 各レースに対して処理を行う
for race_id in df.index:
  df_p = pd.DataFrame(index=[race_id])  # レースごとのデータフレーム
  df_p.index.name = 'race_id'  # race_id をインデックスに設定

  # 各ターゲットカラムに対して処理を行う
  for col in df.columns:
    rank_list = []
    returns_list = []
    values = df.loc[race_id, col].replace("'", '').replace("[", '').replace("]", '').split(',')
    values = [int(x) for x in values]  # 数値に変換
    
    # 値を100以下と100以上に分けて処理
    for n in values:
      if n < 100:
        rank_list.append(int(n))  # 100以下は着順リストへ
      else:
        returns_list.append(int(n))  # 100以上は払い戻しリストへ

    # 新しいカラムを追加
    df_p[f'{col}_rank'] = [rank_list]  # 着順リスト
    df_p[f'{col}_returns'] = [returns_list]  # 払い戻しリスト

  # 結果を結合
  df_a = pd.concat([df_a, df_p])

AttributeError: 'float' object has no attribute 'replace'

In [100]:
df_a

,単勝_rank,単勝_returns,複勝_rank,複勝_returns,枠連_rank,枠連_returns,馬連_rank,馬連_returns,ワイド_rank,ワイド_returns,馬単_rank,馬単_returns,三連複_rank,三連複_returns,三連単_rank,三連単_returns
race_id,,,,,,,,,,,,,,,,
202306010101,[14],[180],"[14, 5, 1]","[110, 150, 480]","[3, 7]",[470],"[5, 14]",[470],"[5, 14, 1, 14, 1, 5]","[220, 790, 1550]","[14, 5]",[660],"[1, 5, 14]",[3040],"[14, 5, 1]",[8890]
202306010102,[8],[1380],"[8, 7, 13]","[460, 440, 400]","[4, 4]",[7010],"[7, 8]",[6420],"[7, 8, 8, 13, 7, 13]","[1930, 2810, 2490]","[8, 7]",[15240],"[7, 8, 13]",[32390],"[8, 7, 13]",[199190]
202306010103,[16],[1590],"[16, 9, 2]","[540, 1000, 440]","[5, 8]",[2580],"[9, 16]",[24290],"[9, 16, 2, 16, 2, 9]","[6100, 3510, 5930]","[16, 9]",[47840],"[2, 9, 16]",[117380],"[16, 9, 2]",[649440]
202306010104,[7],[1590],"[7, 13, 1]","[330, 290, 150]","[4, 7]",[660],"[7, 13]",[6650],"[7, 13, 1, 7, 1, 13]","[1710, 970, 830]","[7, 13]",[13100],"[1, 7, 13]",[7310],"[7, 13, 1]",[78280]
202306010105,[1],[690],"[1, 3, 16]","[180, 150, 170]","[1, 2]",[1130],"[1, 3]",[1220],"[1, 3, 1, 16, 3, 16]","[510, 750, 410]","[1, 3]",[2770],"[1, 3, 16]",[1860],"[1, 3, 16]",[13100]
202306010106,[5],[400],"[5, 2, 7]","[150, 190, 210]","[2, 5]",[1360],"[2, 5]",[1430],"[2, 5, 5, 7, 2, 7]","[520, 590, 690]","[5, 2]",[2170],"[2, 5, 7]",[3220],"[5, 2, 7]",[14150]
202306010107,[11],[1800],"[11, 8, 1]","[460, 210, 640]","[4, 6]",[1270],"[8, 11]",[4520],"[8, 11, 1, 11, 1, 8]","[1660, 3060, 1870]","[11, 8]",[11040],"[1, 8, 11]",[20120],"[11, 8, 1]",[161520]
202306010108,[11],[300],"[11, 9, 7]","[140, 150, 170]","[5, 6]",[490],"[9, 11]",[630],"[9, 11, 7, 11, 7, 9]","[300, 380, 400]","[11, 9]",[1230],"[7, 9, 11]",[1040],"[11, 9, 7]",[3880]
202306010109,[9],[150],"[9, 11, 3]","[110, 160, 190]","[7, 8]",[290],"[9, 11]",[480],"[9, 11, 3, 9, 3, 11]","[260, 350, 550]","[9, 11]",[660],"[3, 9, 11]",[1030],"[9, 11, 3]",[3230]


## 予想

In [9]:
features_22 = pd.read_csv(os.path.join(local_paths.FEATURES_DIR, 'features_2022.csv'), index_col=0,  sep="\t")
features_23 = pd.read_csv(os.path.join(local_paths.FEATURES_DIR, 'features_2023.csv'), index_col=0,  sep="\t")

In [10]:
df_22 = features_22.copy()
df_23 = features_23.copy()

In [56]:
df_22.head()

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,frame,number,sex,age,...,course_len_mode,same_course_len_avg_rank,days_since_last_race,race_interval_category,parent_0,parent_1,parent_2,parent_3,parent_4,parent_5
0,202201010101,2020100678,666,1066,356803,1.0,1,1,0,2,...,1800.0,4.000000,27.0,1.0,467.0,3327.0,174.0,442.0,780.0,3556.0
1,202201010101,2020105743,1091,420,639009,4.0,2,2,0,2,...,1200.0,4.428571,14.0,1.0,216.0,1412.0,195.0,390.0,869.0,4520.0
2,202201010101,2020102526,1170,1005,270006,2.0,3,3,0,2,...,1800.0,4.300000,6.0,0.0,386.0,6962.0,189.0,398.0,515.0,1558.0
3,202201010101,2020102360,1144,431,970006,5.0,4,4,0,2,...,1200.0,5.066667,21.0,1.0,475.0,7902.0,156.0,422.0,667.0,2505.0
4,202201010101,2020102535,1140,1005,270006,3.0,5,5,0,2,...,1800.0,3.250000,13.0,1.0,507.0,8947.0,189.0,457.0,144.0,1241.0


In [57]:
df_23.head()

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,frame,number,sex,age,...,course_len_mode,same_course_len_avg_rank,days_since_last_race,race_interval_category,parent_0,parent_1,parent_2,parent_3,parent_4,parent_5
0,202301010101,2021100648,1095,1128,281008,5.0,1,1,1,2,...,1200.0,4.666667,21.0,1.0,506.0,5537.0,189.0,436.0,584.0,1852.0
1,202301010101,2021100159,1157,1186,170800,6.0,2,2,0,2,...,1200.0,8.000000,14.0,1.0,498.0,6267.0,156.0,462.0,780.0,6337.0
2,202301010101,2021100265,1197,1192,320803,7.0,3,3,1,2,...,1200.0,5.800000,21.0,1.0,318.0,2700.0,189.0,334.0,745.0,6482.0
3,202301010101,2021105553,5339,1160,680031,4.0,4,4,1,2,...,1200.0,4.750000,14.0,1.0,439.0,3258.0,210.0,406.0,741.0,5141.0
4,202301010101,2021101429,1170,1026,425031,1.0,5,5,0,2,...,1200.0,1.600000,14.0,1.0,423.0,5856.0,165.0,329.0,780.0,4573.0


In [60]:
# DistributedDatasetInterface を distribute_lib に置き換える
from tensorflow.python.distribute import distribute_lib

#...

def _is_distributed_dataset(ds):
    return isinstance(ds, distribute_lib.DistributedDataset)

_is_distributed_dataset

<function __main__._is_distributed_dataset(ds)>

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, Flatten, Input


df = df_22.copy()
for col in df.columns:
  if df[col].isnull().sum() > 0:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
      # 数値データの平均値で補完
      df[col].fillna(df[col].mean(), inplace=True)
    else:
      # カテゴリデータは最頻値で補完
      df[col].fillna(df[col].mode()[0], inplace=True)

df['target'] = df['rank'].apply(lambda x: 1 if x <= 2 else 0)  # 馬連、馬単




# 予測に不要なカラムを削除
df_p = df.drop(['race_id', 'rank', 'date', 'reference_date'], axis=1)

# データ分割
X = df_p.drop(['target'], axis=1)
y = df_p['target']

# データ分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# オーバーサンプリング
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# エンベッディングの次元数を指定
embedding_dim = 10  # この値は調整可能

# モデルの構築
model = Sequential()

# Horse IDのエンベッディング
model.add(Embedding(input_dim=len(X['horse_id'].unique()), output_dim=embedding_dim, input_length=1))
model.add(Flatten())  # Flattenすることで1次元のベクトルに変換

# Jockey IDのエンベッディング
model.add(Embedding(input_dim=len(X['jockey_id'].unique()), output_dim=embedding_dim, input_length=1))
model.add(Flatten())

# 全結合層
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 出力層

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# モデルのトレーニング
model.fit(X_train_smote, y_train_smote)

# 予測
y_pred_proba = model.predict(X_test)

y_pred = (y_pred_proba >= 0.5).astype(int)

C:\Users\SV8\AppData\Local\Temp\ipykernel_10688\602671004.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
C:\Users\SV8\AppData\Local\Temp\ipykernel_10688\602671004.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

AttributeError: module 'tensorflow.python.distribute.input_lib' has no attribute 'DistributedDatasetInterface'

In [6]:
model_0 = 1
model_1 = 2
models = [model_0, model_1]
for model in models:
  i = re.findall(r'\d+', {model})
  print(i)


['1']
['2']


In [46]:
for col in df_23.columns:
  if df_23[col].isnull().sum() > 0:
    if df_23[col].dtype == 'float64' or df_23[col].dtype == 'int64':
      # 数値データの平均値で補完
      df_23[col].fillna(df_23[col].mean(), inplace=True)
    else:
      # カテゴリデータは最頻値で補完
      df_23[col].fillna(df_23[col].mode()[0], inplace=True)

C:\Users\SV8\AppData\Local\Temp\ipykernel_10688\2937735567.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_23[col].fillna(df_23[col].mean(), inplace=True)
C:\Users\SV8\AppData\Local\Temp\ipykernel_10688\2937735567.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [47]:
df_23

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,frame,number,sex,age,...,course_len_mode,same_course_len_avg_rank,days_since_last_race,race_interval_category,parent_0,parent_1,parent_2,parent_3,parent_4,parent_5
0,202301010101,2021100648,1095,1128,281008,5.0,1,1,1,2,...,1200.0,4.666667,21.0,1.0,506.0,5537.0,189.0,436.0,584.0,1852.0
1,202301010101,2021100159,1157,1186,170800,6.0,2,2,0,2,...,1200.0,8.000000,14.0,1.0,498.0,6267.0,156.0,462.0,780.0,6337.0
2,202301010101,2021100265,1197,1192,320803,7.0,3,3,1,2,...,1200.0,5.800000,21.0,1.0,318.0,2700.0,189.0,334.0,745.0,6482.0
3,202301010101,2021105553,5339,1160,680031,4.0,4,4,1,2,...,1200.0,4.750000,14.0,1.0,439.0,3258.0,210.0,406.0,741.0,5141.0
4,202301010101,2021101429,1170,1026,425031,1.0,5,5,0,2,...,1200.0,1.600000,14.0,1.0,423.0,5856.0,165.0,329.0,780.0,4573.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47268,202310030812,2019102542,1208,1161,17803,1.0,6,8,1,4,...,1800.0,5.195876,29.0,1.0,251.0,517.0,189.0,301.0,618.0,1738.0
47269,202310030812,2020103754,1116,1104,651031,6.0,7,9,0,3,...,1800.0,5.966667,28.0,1.0,299.0,6949.0,81.0,337.0,384.0,1551.0
47270,202310030812,2020100400,1193,1092,180800,4.0,7,10,1,3,...,1800.0,8.307692,21.0,1.0,236.0,5649.0,176.0,288.0,780.0,6282.0
47271,202310030812,2020103663,732,1092,226800,2.0,8,11,2,3,...,1800.0,6.142857,91.0,3.0,236.0,8117.0,176.0,288.0,185.0,1206.0


In [48]:
for id in ['horse_id', 'trainer_id', 'jockey_id', 'owner_id']:
  le = LabelEncoder()
  df_23[f'{id}_encoded'] = le.fit_transform(df_23[f'{id}'])

# 予測に不要なカラムを削除
df_23_x = df_23.drop(['race_id', 'rank', 'horse_id', 'trainer_id', 'jockey_id', 'owner_id', 'date', 'reference_date'], axis=1)

In [49]:
# 予測
proba = model.predict_proba(df_23_x)[:, 1]

# しきい値を設定して予測を0, 1に変換
predicted_target = (proba >= 0.5).astype(int)

In [39]:
df_23['predicted_target'] = predicted_target

In [41]:
df_23['predicted_target'].value_counts()

predicted_target
0    30630
1    16643
Name: count, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_shuffled = df_22.sample(frac=1, random_state=1).reset_index(drop=True)
df_t, val_or_pred = train_test_split(df_shuffled, test_size=0.2, random_state=1)

In [15]:
df_t.isnull().sum()

race_id                        0
horse_id                       0
jockey_id                      0
trainer_id                     0
owner_id                       0
rank                           0
frame                          0
number                         0
sex                            0
age                            0
impost                         0
win_odds                       0
popularity                     0
weight                         0
weight_diff                    0
date                           0
race_type                      0
around                      1318
course_len                   266
weather                      255
ground_state                 255
race_class                  5919
place                       1882
reference_date              3761
rank_mean                   3761
rank_min                    3761
rank_max                    3761
rank_std                    7210
n_horses_mean               3761
rank_diff_mean              3765
rank_diff_

## 馬の血統データの追加

In [110]:
results = pd.read_csv(os.path.join(local_paths.RAW_DIR, 'results_2022.csv'), index_col=0 ,sep="\t")
results

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id,trainer_id,owner_id
race_id,,,,,,,,,,,,,,,,,
202206010101,1,8,15,ニシノアナ,牝3,51.0,横山琉人,1:12.5,NaN,6.8,4.0,456(+4),[東] 相沢郁,2019103610,1192,1020,897009
202206010101,2,5,10,トラストパッキャオ,牝3,54.0,菅原明良,1:12.5,クビ,57.2,12.0,458(+2),[東] 高木登,2019100855,1179,1088,125008
202206010101,3,2,4,マイネルシトラス,牡3,56.0,柴田大知,1:12.5,クビ,3.7,1.0,518(-2),[東] 武市康男,2019103542,1009,1089,546800
202206010101,4,1,2,ピカリエ,牝3,54.0,伊藤工真,1:12.8,1.1/2,16.0,9.0,486(+6),[東] 金成貴史,2019104288,1119,1132,64803
202206010101,5,8,16,ブラッドライン,牡3,56.0,Ｍ．デム,1:13.2,2.1/2,10.0,5.0,478(-2),[東] 伊藤大士,2019101003,5212,1109,427030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202209060912,12,6,12,モンファボリ,牝4,53.0,吉田隼人,1:09.5,1/2,22.2,9.0,452(0),[西] 須貝尚介,2018105223,1095,1105,226800
202209060912,13,3,6,テイエムイダテン,牡5,54.0,松田大作,1:09.8,1.3/4,65.4,12.0,484(+4),[西] 飯田雄三,2017102603,1030,1050,356002
202209060912,14,4,8,ショウナンアリアナ,牝6,52.0,西村淳也,1:09.8,アタマ,153.7,16.0,452(+2),[西] 高野友和,2016110041,1171,1117,331002


In [134]:
horse_id_list_22 = list(results['horse_id'].unique())

In [135]:
html_paths_horse = []
for horse_id in horse_id_list_22:
  html_paths_horse.append(os.path.join(local_paths.HTML_HORSE_DIR, str(horse_id) + '.bin'))

In [136]:
html_paths_horse

['c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019103610.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019100855.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019103542.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019104288.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019101003.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019101217.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019101408.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019102271.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019104028.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019105170.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019102173.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019102153.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019101943.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019105024.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019104143.b

In [137]:
len(html_paths_horse)

11557

In [143]:
html_paths_horse[:10]

['c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019103610.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019100855.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019103542.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019104288.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019101003.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019101217.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019101408.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019102271.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019104028.bin',
 'c:\\Users\\SV8\\m\\p\\keiba\\data\\html\\horse\\2019105170.bin']

In [200]:
dfs = {}
for html_path in html_paths_horse[:10]:
  with open(html_path, "rb") as f:
    try:
      horse_id = re.search(r'\d{10}', html_path).group()
      html = f.read()
      df = pd.read_html(html)[2]

      df['horse_id'] = [horse_id] * len(df)

    except IndexError as e:
      print(f"table not found at {horse_id}")
      continue

  concat_df = pd.concat(dfs.values())
  concat_df.index = concat_df['horse_id']
  concat_df = concat_df.drop(['horse_id'], axis=1)

ValueError: No objects to concatenate

In [198]:
dfs

{'2019103610':           0            1    horse_id
 0     ドレフォン    Gio Ponti  2019103610
 1     ドレフォン     Eltimaas  2019103610
 2  レッドセイリング      ゼンノロブロイ  2019103610
 3  レッドセイリング  チャールストンハーバー  2019103610,
 '2019100855':          0                1    horse_id
 0  ヘニーヒューズ             ヘネシー  2019100855
 1  ヘニーヒューズ     Meadow Flyer  2019100855
 2      アリー  Deputy Minister  2019100855
 3      アリー   Secret Partner  2019100855,
 '2019103542':           0          1    horse_id
 0  ジョーカプチーノ  マンハッタンカフェ  2019103542
 1  ジョーカプチーノ     ジョープシケ  2019103542
 2   マイネオレンジ   ロージズインメイ  2019103542
 3   マイネオレンジ   コスモバレンシア  2019103542,
 '2019104288':          0          1    horse_id
 0  エイシンヒカリ  ディープインパクト  2019104288
 1  エイシンヒカリ      キャタリナ  2019104288
 2   バンデリータ  メイショウオウドウ  2019104288
 3   バンデリータ   リトルバンデーラ  2019104288,
 '2019101003':           0          1    horse_id
 0  ローレルゲレイロ    キングヘイロー  2019101003
 1  ローレルゲレイロ    ビッグテンビー  2019101003
 2  フルオブスターズ  エンパイアメーカー  2019101003
 3  フルオブスターズ  ポエティクシーズン  2019101

In [201]:
concat_df

,0,1
horse_id,,
2019103610,ドレフォン,Gio Ponti
2019103610,ドレフォン,Eltimaas
2019103610,レッドセイリング,ゼンノロブロイ
2019103610,レッドセイリング,チャールストンハーバー
2019100855,ヘニーヒューズ,ヘネシー
2019100855,ヘニーヒューズ,Meadow Flyer
2019100855,アリー,Deputy Minister
2019100855,アリー,Secret Partner
2019103542,ジョーカプチーノ,マンハッタンカフェ


In [203]:
# horse_idごとに親の情報を整理
result = []

# horse_idごとにデータを集約
for horse_id, group in concat_df.groupby('horse_id'):
  # 親の情報を取得
  parents = group[0].tolist()  
  parents = list(dict.fromkeys(parents))  # 親
  grandparents = group[1].tolist()  # 祖父母
  
  # 合併
  row = {
      'horse_id': horse_id,
      'parent_0': parents[0] if len(parents) > 0 else None,
      'parent_1': parents[1] if len(parents) > 1 else None,
      'parent_2': grandparents[0] if len(grandparents) > 0 else None,
      'parent_3': grandparents[1] if len(grandparents) > 1 else None,
      'parent_4': grandparents[2] if len(grandparents) > 2 else None,
      'parent_5': grandparents[3] if len(grandparents) > 3 else None,
  }
  result.append(row)

# 新しいDataFrameを作成
bloodline_df = pd.DataFrame(result)


In [204]:
bloodline_df

,horse_id,parent_0,parent_1,parent_2,parent_3,parent_4,parent_5
0,2019100855,ヘニーヒューズ,アリー,ヘネシー,Meadow Flyer,Deputy Minister,Secret Partner
1,2019101003,ローレルゲレイロ,フルオブスターズ,キングヘイロー,ビッグテンビー,エンパイアメーカー,ポエティクシーズン
2,2019101217,エスポワールシチー,ビヨンドマイリーチ,ゴールドアリュール,エミネントシチー,ワイルドラッシュ,シーズプリンセス
3,2019101408,ハービンジャー,リップル,Dansili,Penang Pearl,アグネスタキオン,レインボーリップル
4,2019102271,キンシャサノキセキ,ミルテ,フジキセキ,ケルトシャーン,アフリート,カモンマイハウス
5,2019103542,ジョーカプチーノ,マイネオレンジ,マンハッタンカフェ,ジョープシケ,ロージズインメイ,コスモバレンシア
6,2019103610,ドレフォン,レッドセイリング,Gio Ponti,Eltimaas,ゼンノロブロイ,チャールストンハーバー
7,2019104028,ドレフォン,アイアムネフライト,Gio Ponti,Eltimaas,アグネスタキオン,アイアムアドーター
8,2019104288,エイシンヒカリ,バンデリータ,ディープインパクト,キャタリナ,メイショウオウドウ,リトルバンデーラ
9,2019105170,トゥザワールド,クルージンミジー,キングカメハメハ,トゥザヴィクトリー,Mizzen Mast,Baby Lets Cruise


In [171]:
# horse_idごとにデータを集約
for horse_id, group in concat_df.groupby('horse_id'):
    # 親の情報を取得
    parents = group[0].tolist()  # 親（カラム0）
    parents = list(dict.fromkeys(parents))
    grandparents = group[1].tolist()  # 祖父母（カラム1）

In [172]:
parents

['トゥザワールド', 'クルージンミジー']

In [174]:
# horse_idごとに親の情報を整理
result = []

# horse_idごとにデータを集約
for horse_id, group in concat_df.groupby('horse_id'):
    # 親の情報を取得
    parents = group[0].tolist()  
    parents = list(dict.fromkeys(parents))  # 親
    grandparents = group[1].tolist()  # 祖父母
    
    # 合併
    row = {
        'horse_id': horse_id,
        'parent_0': parents[0] if len(parents) > 0 else None,
        'parent_1': parents[1] if len(parents) > 1 else None,
        'parent_2': grandparents[0] if len(grandparents) > 0 else None,
        'parent_3': grandparents[1] if len(grandparents) > 1 else None,
        'parent_4': grandparents[2] if len(grandparents) > 2 else None,
        'parent_5': grandparents[3] if len(grandparents) > 3 else None,
    }
    result.append(row)

# 新しいDataFrameを作成
bloodline_df = pd.DataFrame(result)
bloodline_df

,horse_id,parent_0,parent_1,parent_2,parent_3,parent_4,parent_5
0,2019100855,ヘニーヒューズ,アリー,ヘネシー,Meadow Flyer,Deputy Minister,Secret Partner
1,2019101003,ローレルゲレイロ,フルオブスターズ,キングヘイロー,ビッグテンビー,エンパイアメーカー,ポエティクシーズン
2,2019101217,エスポワールシチー,ビヨンドマイリーチ,ゴールドアリュール,エミネントシチー,ワイルドラッシュ,シーズプリンセス
3,2019101408,ハービンジャー,リップル,Dansili,Penang Pearl,アグネスタキオン,レインボーリップル
4,2019102271,キンシャサノキセキ,ミルテ,フジキセキ,ケルトシャーン,アフリート,カモンマイハウス
5,2019103542,ジョーカプチーノ,マイネオレンジ,マンハッタンカフェ,ジョープシケ,ロージズインメイ,コスモバレンシア
6,2019103610,ドレフォン,レッドセイリング,Gio Ponti,Eltimaas,ゼンノロブロイ,チャールストンハーバー
7,2019104028,ドレフォン,アイアムネフライト,Gio Ponti,Eltimaas,アグネスタキオン,アイアムアドーター
8,2019104288,エイシンヒカリ,バンデリータ,ディープインパクト,キャタリナ,メイショウオウドウ,リトルバンデーラ
9,2019105170,トゥザワールド,クルージンミジー,キングカメハメハ,トゥザヴィクトリー,Mizzen Mast,Baby Lets Cruise


In [164]:
df_pivot

,horse_id,parent_0,parent_1,parent_2,parent_3,parent_4,parent_5,parent_6,parent_7
0,2019100855,ヘニーヒューズ,ヘニーヒューズ,アリー,アリー,ヘネシー,Meadow Flyer,Deputy Minister,Secret Partner
1,2019101003,ローレルゲレイロ,ローレルゲレイロ,フルオブスターズ,フルオブスターズ,キングヘイロー,ビッグテンビー,エンパイアメーカー,ポエティクシーズン
2,2019101217,エスポワールシチー,エスポワールシチー,ビヨンドマイリーチ,ビヨンドマイリーチ,ゴールドアリュール,エミネントシチー,ワイルドラッシュ,シーズプリンセス
3,2019101408,ハービンジャー,ハービンジャー,リップル,リップル,Dansili,Penang Pearl,アグネスタキオン,レインボーリップル
4,2019102271,キンシャサノキセキ,キンシャサノキセキ,ミルテ,ミルテ,フジキセキ,ケルトシャーン,アフリート,カモンマイハウス
5,2019103542,ジョーカプチーノ,ジョーカプチーノ,マイネオレンジ,マイネオレンジ,マンハッタンカフェ,ジョープシケ,ロージズインメイ,コスモバレンシア
6,2019103610,ドレフォン,ドレフォン,レッドセイリング,レッドセイリング,Gio Ponti,Eltimaas,ゼンノロブロイ,チャールストンハーバー
7,2019104028,ドレフォン,ドレフォン,アイアムネフライト,アイアムネフライト,Gio Ponti,Eltimaas,アグネスタキオン,アイアムアドーター
8,2019104288,エイシンヒカリ,エイシンヒカリ,バンデリータ,バンデリータ,ディープインパクト,キャタリナ,メイショウオウドウ,リトルバンデーラ
9,2019105170,トゥザワールド,トゥザワールド,クルージンミジー,クルージンミジー,キングカメハメハ,トゥザヴィクトリー,Mizzen Mast,Baby Lets Cruise


In [149]:


# カラム名の整理
pivot_df.columns = ['horse_id', '父馬_祖父母', '母馬_祖父母']

# horse_idごとに親馬の情報をまとめる
parents_pivot_df = concat_df.pivot_table(index='horse_id', columns=0, values=0, aggfunc='first').reset_index()

# カラム名の整理
parents_pivot_df.columns = ['horse_id', '父馬', '母馬']

# 親馬のデータと祖父母のデータを統合
bloodline_combined_df = parents_pivot_df.merge(pivot_df, on='horse_id', how='left')
bloodline_combined_df

ValueError: Length mismatch: Expected axis has 20 elements, new values have 3 elements